# Initalize libraries

## Import libraries

In [ ]:
import sys, os
import time
from os.path import join
from os import path
from importlib import reload
from getpass import getuser
from glob import glob
from tqdm.auto import tqdm

# Data
import xarray as xr
import h5py
import numpy as np
from nexusformat.nexus import *

# Plotting
import imageio
import matplotlib.pyplot as plt
import fabio
import skimage.morphology

# skimage
from skimage.draw import ellipse

# scipy
from scipy.ndimage.filters import gaussian_filter
import scipy
from scipy import stats

# pyFAI
import pyFAI
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
from pyFAI.detectors import Detector

# Self-written libraries
sys.path.append(join(os.getcwd(), "library"))
import reconstruct as reco
import fthcore as fth
import support_functions as sup
import interactive
from interactive import cimshow
import reconstruct_rb as rec
import reconstruct as reco

plt.rcParams["figure.constrained_layout.use"] = True  # replaces plt.tight_layout

In [ ]:
# Is there a GPU?
try:
    # Cupy
    import cupy as cp
    import cupyx as cpx

    GPU = True

    print("GPU available")

    # Self-written library
    import CCI_core_cupy as cci
    import Phase_Retrieval as PhR
except:
    GPU = False
    import CCI_core as cci

    print("GPU unavailable")

In [ ]:
# interactive plotting
import ipywidgets

%matplotlib widget

# Auto formatting of cells
#%load_ext jupyter_black

## Experiment specific Functions

In [ ]:
PROPOSAL = 11018955
USER = getuser()

### Loading data

In [ ]:
BASEFOLDER = "/asap3/petra3/gpfs/p04/2024/data/%s/" % PROPOSAL
sample_name = "2403_tomo"


# Load any kind of data from collection
def load_pre_scan_snapshot(scan_id, key):
    fname = join(BASEFOLDER, "raw", "%s_%05d.h5" % (sample_name,scan_id))
    entry = "entry%d"%scan_id
    
    with h5py.File(fname, "r") as f:
        data = np.array(f[entry]["measurement"]["pre_scan_snapshot"][key][()])
        print("Loaded: %s" % (fname))
    return np.squeeze(data)

# Load any kind of data from measurements
def load_data(scan_id, key):
    fname = join(BASEFOLDER, "raw", "%s_%05d.h5" % (sample_name,scan_id))
    entry = "entry%d"%scan_id

    with h5py.File(fname, "r") as f:
        data = np.array(f[entry]["measurement"][key][()])
        print("Loaded key %s of %s" % (key, entry))
    return np.squeeze(data)


def get_image_id(im_id):
    fname = join(BASEFOLDER, "raw", "%s_%05d.h5" % (sample_name,im_id))
    entry = "entry%d"%im_id

    with h5py.File(fname, "r") as f:
        spe_id = str(f[entry]["measurement"]["ccd2"][()])[-10:-6]
    return spe_id

def load_spe_files(spe_id):
    fname = join(BASEFOLDER, "raw", sample_name +"  " +str(spe_id) + ".spe")
    im_out = np.array(imageio.mimread(fname, memtest="5000MB"))

    return im_out
    
# Load image files
def load_images(im_id):
    """
    Load ccd images from nxs files
    """
    
    spe_id = get_image_id(im_id)
    im_out = load_spe_files(spe_id)
    
    return im_out


# Full image loading procedure
def load_processing(im_id, crop=None):
    """
    Loads images, averaging of two individual images (scans in tango consist of two images),
    padding to square shape, Additional cropping (optional)
    """

    # Load data
    images = load_images(im_id)

    # Zeropad to get square shape
    #images = sup.padding(images)

    # Calculate mean
    if images.ndim > 2:
        image = np.mean(images, axis=0)
    else:
        image = images.copy()

    # Optional cropping
    if crop is not None:
        images = images[:, :crop, :crop]
        image = image[:crop, :crop]

    return image

### Loading, saving fth & cdi data

In [ ]:
# Saving of log files for fth and cdi recos
def save_fth_h5():
    # Save h5
    data = {}
    data["im_id"] = im_id
    data["topo_id"] = topo_id
    data["topo_centered"] = topo_c
    data["im_centered"] = im_c
    data["holo"] = holo
    data["sum_b"] = sum_b
    data["recon"] = recon
    data["factor"] = factor
    data["offset"] = offset
    data["center"] = center
    data["roi"] = roi
    data["prop_dist"] = prop_dist
    data["phase"] = phase
    data["mask_bs"] = mask_bs
    data["bs_diam"] = bs_diam
    data["experimental_setup"] = experimental_setup

    filename = join(
        folder_general, "Logs", "Data_ImId_%04d_RefId_%s_%s" % (im_id, topo_id, USER)
    )
    print("Now Saving: %s" % filename)
    cci.create_hdf5(data, filename)


def save_cdi_h5():
    # Save h5
    data = {}
    data["im_id"] = im_id
    data["topo_id"] = topo_id
    data["pos"] = pos
    data["neg"] = neg
    data["factor"] = factor
    data["offset"] = offset
    data["center"] = center
    data["roi"] = roi
    data["prop_dist"] = prop_dist_cdi
    data["phase"] = phase_cdi
    data["mask_bs"] = mask_bs_cdi
    data["bs_diam"] = bs_diam_cdi
    data["supportmask"] = supportmask
    data["mask_pixel"] = mask_pixel
    data["p"] = p
    data["n"] = n
    data["p_pc"] = p_pc
    data["n_pc"] = n_pc
    data["gamma_p"] = gamma_p
    data["gamma_n"] = gamma_n
    data["experimental_setup"] = experimental_setup

    filename = join(
        folder_general,
        "Logs",
        "Data_ImId_%04d_RefId_%s_cdi_%s" % (im_id, topo_id, USER),
    )
    print("Now Saving: %s" % filename)
    cci.create_hdf5(data, filename)
    return


def save_topo_holo(topo_c, pos_id, neg_id):
    """
    Save only topo holos which can be later used for single helicity reconstructions
    """
    data = {}
    data["pos_id"] = pos_id
    data["neg_id"] = neg_id
    data["topo"] = topo_c

    filename = join(
        folder_general,
        "Topos",
        "Topo_ImId_%04d_RefId_%04d_cdi_%s" % (pos_id, neg_id, USER),
    )
    print("Now Saving: %s" % filename)
    cci.create_hdf5(data, filename)
    return


def load_topo_holo(pos_id, neg_id):
    """
    Load topo holos for single helicity reconstructions
    """
    fname = join(
        folder_general,
        "Topos",
        "Topo_ImId_%04d_RefId_%04d_cdi_%s.hdf5" % (pos_id, neg_id, USER),
    )

    with h5py.File(fname, "r") as f:
        im_out = f["topo"][()]
    return im_out


def load_cdi(im_id, topo_id):
    """
    Load cdi dataset
    """
    fname = join(
        folder_general,
        "Logs",
        "Data_ImId_%04d_RefId_%s_cdi_%s.hdf5" % (im_id, topo_id, USER),
    )

    with h5py.File(fname, "r") as f:
        data = {}
        data["im_id"] = f["im_id"][()]
        data["topo_id"] = f["topo_id"][()]
        data["pos"] = f["pos"][()]
        data["neg"] = f["neg"][()]
        data["recon"] = f["recon"][()]
        data["factor"] = f["factor"][()]
        data["offset"] = f["offset"][()]
        data["shift"] = f["shift"][()]
        data["center"] = f["center"][()]
        data["prop_dist"] = f["prop_dist"][()]
        data["phase"] = f["phase"][()]
        data["mask_bs"] = f["mask_bs"][()]
        data["supportmask"] = f["supportmask"][()]
        data["mask_pixel"] = f["mask_pixel"][()]
        data["p"] = f["p"][()]
        data["n"] = f["n"][()]
        data["p_pc"] = f["p_pc"][()]
        data["n_pc"] = f["n_pc"][()]
        #data["srotx"] = f["experimental_setup"]["srotx"][()]
        data["srotz"] = f["experimental_setup"]["srotz"][()]
    return data_cdi

### Masking

In [ ]:
from matplotlib.path import Path


def create_single_polygon_mask(shape, coordinates):
    """
    Creates a polygon mask from coordinates of corner points

    Parameter
    =========
    shape : int tuple
        shape/dimension of output array
    coordinates: nested list
        coordinates of polygon corner points [(yc_1,xc_1),(yc_2,xc_2),...]


    Output
    ======
    mask: array
        binary mask where filled polygon is "1"
    ======
    author: ck 2023
    """

    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]))
    x, y = x.flatten(), y.flatten()

    points = np.vstack((x, y)).T

    path = Path(coordinates)
    mask = path.contains_points(points)
    mask = mask.reshape(shape)
    return mask


def create_polygon_mask(shape, coordinates):
    """
    Creates multiple polygon masks from set of coordinates of corner points

    Parameter
    =========
    shape : int tuple
        shape/dimension of output array
    coordinates: nested list
        coordinates of polygon corner points for multiple polygons
        [[(yc_1,xc_1),(yc_2,xc_2),...],[(yc_1,xc_1),(yc_2,xc_2),...]]

    Output
    ======
    mask: array
        binary mask where filled polygons are "1"
    ======
    author: ck 2023
    """

    if len(coordinates) == 1:
        mask = create_single_polygon_mask(shape, coordinates[0])

    # Loop over coordinates
    elif len(coordinates) > 1:
        mask = np.zeros(shape)
        for coord in coordinates:
            mask = mask + create_single_polygon_mask(shape, coord)
            mask[mask > 1] = 1

    return mask


def load_poly_masks(shape, polygon_name_list):
    """
    Loads set of polygon masks based on stored coordinates

    Parameter
    =========
    shape : tuple
        shape of output mask
    polygon_name_list : list
        keys of different mask coordinates to load

    Output
    ======
    mask: array
        binary mask where filled polygons are "1"
    ======
    author: ck 2023
    """

    mask = []

    # Load dictionary of coordinates
    mask_coordinates = load_poly_coordinates()

    # Loop over relevant mask keys
    for polygon_name in polygon_name_list:
        coord = mask_coordinates[polygon_name]
        mask.append(create_polygon_mask(im_c.shape, coord).astype(float))

    # Combine all individual mask layers
    mask = np.array(mask)
    mask = np.sum(mask, axis=0)
    mask[mask > 1] = 1

    return mask

In [ ]:
def auto_shift_mask(
    mask,
    image,
    shift_range_y=(-10, 10),
    shift_range_x=(-10, 10),
    step_size=1,
    crop=None,
    method="minimize",
):
    """
    Automatically shifts a binary (beamstop) mask to the optimal position

    Parameter
    =========
    mask : array
        binary mask, e.g., beamstop mask
    image : array
        image partially covered by mask
    shift_range_y, _x : tupel
        Min and max limit for search area in y- and x-direction
    step_size : scalar
        step size of scan area
    crop : int
        additional cropping of arrays to speed up calculations
        Crops according to [crop:-crop,crop:-crop]
    method : string
        method used for optimization. 'minimize' or 'maximize' metric <mask,image>

    Output
    ======
    optimized shift: tupel
        shift vector for optimized posistion
    mask_shifted: array
        mask shifted according to best shift vector
    overlap : array
        computed evaluation metric
    ======
    author: ck 2023
    """

    # Basic looping
    # Create set of arrays for shifting
    yshift = np.arange(shift_range_y[0], shift_range_y[1], step_size)
    xshift = np.arange(shift_range_x[0], shift_range_x[1], step_size)

    # Setup loss function to evaluate overlap
    overlap = np.zeros((yshift.shape[0], xshift.shape[0]))

    # Optinal cropping to reduce computation time
    if crop is not None:
        tmask, timage = mask[crop:-crop, crop:-crop], image[crop:-crop, crop:-crop]
    else:
        tmask, timage = mask.copy(), timage.copy()

    # Loop over all combinations of shifts
    for i, y in enumerate(yshift):
        for j, x in enumerate(xshift):
            # Shift mask to new position
            mask_shift = cci.shift_image(tmask, [y, x])

            # Calculate overlap
            overlap[i, j] = np.sum(mask_shift * timage)

    # Get best shift
    if method == "minimize":
        idx = np.unravel_index(
            np.argmin(
                overlap,
            ),
            overlap.shape,
        )
    elif method == "maximize":
        idx = np.unravel_index(
            np.argmax(
                overlap,
            ),
            overlap.shape,
        )

    # Output
    optimized_shift = (yshift[idx[0]], yshift[idx[1]])
    print(
        "Best mask overlap for shift: [%.2f,%.2f]"
        % (optimized_shift[0], optimized_shift[1])
    )

    # Shift
    mask_shifted = cci.shift_image(mask, optimized_shift)

    # Binarize mask (necessary for sub-px shift)
    mask_shifted[mask_shifted > 0.5] = 1
    mask_shifted[mask_shifted < 0.5] = 0

    return optimized_shift, mask_shifted, overlap

In [ ]:
def create_supportmask(support_coordinates, shape):
    """
    Create cdi support mask from a combination of multiple circular apertures

    Parameter
    =========
    support_coordinates: nested list
        Contains center coordinates and radius of each aperture [[yc_1,xc_1,r_1],[yc_2,xc_2,r_2],...]
    shape : int tuple
        shape/dimension of output array

    Output
    ======
    supportmask: array
        composed binary mask where circular apertures are "1"
    ======
    author: ck 2023

    """

    # Create support mask
    supportmask = np.zeros(shape)
    for i in range(len(support_coordinates)):
        supportmask += cci.circle_mask(
            supportmask.shape,
            [support_coordinates[i][0], support_coordinates[i][1]],
            support_coordinates[i][2],
        )

    return supportmask

In [ ]:
def create_ellipse_supportmask(support_coordinates, shape):
    """
    Create cdi support mask as a combination of multiple ellipses

    Parameter
    =========
    support_coordinates: nested list
        Contains center coordinates, height, width and rotation angle of each aperture 
        [[(yc_1,xc_1),height_1,width_1,angle_1],[(yc_2,xc_2),height_2,width_2,angle_2],...]
    shape : int tuple
        shape/dimension of output array

    Output
    ======
    supportmask: array
        composed binary mask where ellipses apertures are "1"
    ======
    author: ck/sg 2024

    """

    # Create support mask
    supportmask = np.zeros(shape)
    for i in range(len(support_coordinates)):
        center, height, width, angle = support_coordinates[i][0], support_coordinates[i][1], support_coordinates[i][2], support_coordinates[i][3]
        yy, xx = ellipse(center[1],center[0],height/2,width/2,rotation=-angle) 
        supportmask[yy,xx] = 1
    
    return supportmask

### Other

In [ ]:
def differential_operator(shape,center, experimental_setup, angle = 0):
    '''
    Calculates Fourier-space differential operator for heraldo reconstruction
    
    Parameter
    ---------
    shape: int tuple
        shape of output array
    center: tuple
        array center coordinates (y,x) 
    experimental_setup: dict
        must contain detector pixel_size ["px_size"], distance ["ccd_dist"],
        wavelength ["lambda"]
    angle: float
        rotation angle of heraldo slit
        
    Returns:
    --------
    return: complex array
        differential operator in Fourier space
        
    '''
    
    # Convert deg to rad
    angle = np.deg2rad(angle)
    
    # Create x,y grid to convert pixel in q-space
    x, y = np.meshgrid(np.arange(shape[0]),np.arange(shape[0]))
    
    # Center meshgrid
    y, x = y - center[0],  x - center[1]
    
    # Multiplay with pixel size
    y, x = y*experimental_setup["px_size"], x*experimental_setup["px_size"]
    
    # Convert to q-space
    qy = 4*np.pi/experimental_setup["lambda"] * np.sin(0.5*np.arctan(y/experimental_setup["ccd_dist"]))
    qx = 4*np.pi/experimental_setup["lambda"] * np.sin(0.5*np.arctan(x/experimental_setup["ccd_dist"]))
    
    # Normalize q space to [-1,1]
    qy, qx = qy/np.max(np.abs(qy)), qx/np.max(np.abs(qx))
    
    return 2j*np.pi*qx*np.cos(angle) + 2j*np.pi*qy*np.sin(angle)

In [ ]:
def reconstruct_heraldo(holo, experimental_setup, center = None, prop_dist = 0, phase = 0,angle = 0):
    '''
    Reconstruction of holograms in heraldo reference scheme
    
    Parameter:
    ----------
    holo: array
        Centered input hologram
    experimental_setup: dict
        must contain detector pixel_size ["px_size"], distance ["ccd_dist"],
        wavelength ["lambda"]
    center: tuple or None
        array center coordinates (y,x) 
    prop_dist: float
        propagation distance
    phase: float
        global phase shift of complex array
    angle: float
        rotation angle of heraldo slit
        
    returns:
    image: array
        reconstructed image
    heraldo_operator: complex array
        differential operator in Fourier space
    '''
    
    if center is None:
        center = np.array(holo.shape)/2
    
    heraldo_operator = differential_operator(holo.shape,center, 
                    experimental_setup, angle = angle)
    holo = holo * heraldo_operator
    holo = fth.propagate(holo, prop_dist*1e-6, experimental_setup = experimental_setup)*np.exp(1j*phase)
    image = fth.reconstruct(holo)
    
    return image, heraldo_operator

In [ ]:
# Worker which performs complete fth reconstruction process
def worker(image, topo):
    # Centering
    shift_c = np.array(topo.shape) / 2 - center
    topo_c = cci.shift_image(topo, shift_c)
    im_c = cci.shift_image(image, shift_c)
    
    ## Image registration
    #shift = cci.image_registration(
    #    im_c[roi_im_reg],
    #    topo_c[roi_im_reg],
    #    method="dipy",
    #)
    shift = [0,0]
    print("Relative shift is: %s"%shift)

    
    # Correct relative drift
    topo_c = cci.shift_image(topo_c, shift) 
    
    # Optional ewalds sphere projection
    if project_ewalds_sphere is True:
        %%time
        proj_im = PhR.inv_gnomonic2(im_c,alpha=experimental_setup["angle"], center=None, experimental_setup = experimental_setup, method='cubic' , mask=None)
        proj_topo = PhR.inv_gnomonic2(topo_c,alpha=experimental_setup["angle"], center=None, experimental_setup = experimental_setup, method='cubic' , mask=None)

        im_c = proj_im.copy()
        topo_c = proj_topo.copy()

    # Create masks
    # Load polygon mask and shift to manually determined position
    mask = load_poly_masks(im_c.shape, polygon_names)
    mask = np.round(cci.shift_image(mask,mask_shift))
    
    # Increase/Decrease mask size
    footprint = skimage.morphology.disk(np.abs(mask_scale))
    if mask_scale > 0:
        mask = skimage.morphology.dilation(mask, footprint) # increase size
    elif mask_scale < 0: 
        mask = skimage.morphology.erosion(mask, footprint) # decrease size

    # Optimize position of drawn mask relative to target image
    if optimize_position is True:
        # level 1 (rough)
        optimized_shift, _, _ = auto_shift_mask(
            mask,
            im_c,
            shift_range_y=[-10, 10],
            shift_range_x=[-10, 10],
            step_size=2,
            crop=300,
        )
        # Level 2 (fine)
        optimized_shift, mask_shifted, overlap = auto_shift_mask(
            mask,
            im_c,
            shift_range_y=[optimized_shift[0] - 4, optimized_shift[0] + 4],
            shift_range_x=[optimized_shift[1] - 4, optimized_shift[1] + 4],
            step_size=0.5,
            crop=300,
        )
    
    # BS center part
    mask_bs_center = cci.circle_mask(
        topo.shape, np.array(topo.shape) / 2, bs.rBS-3, sigma=None
    )

    # Create image specific beamstop mask
    mask_im = mask.copy()
    mask_im = mask_im + (im_c > 40000) + mask_bs_center
    mask_im[mask_im > 1] = 1

    # Create topo specific beamstop mask
    mask_topo = mask.copy()
    mask_topo = mask_topo + (topo_c > 40000) + mask_bs_center

    # Combine both
    mask_pixel = mask_im + mask_topo
    mask_pixel[mask_pixel > 1] = 1
    
    # Create smooth mask
    footprint = skimage.morphology.disk(6)
    mask_pixel_smooth = skimage.morphology.dilation(mask_pixel, footprint)
    mask_pixel_smooth = gaussian_filter(mask_pixel_smooth, 2)

    # Get scaling factor and offset
    factor, offset = cci.dyn_factor(
        im_c * (1 - mask_pixel),
        topo_c * (1 - mask_pixel),
        method="correlation",
        print_out=False,
        plot=False,
    )

    # Calculate differences (magnetic) and sums (topographc) contrast holograms.
    # _c: centered, without beamstop, _b: centered, with beamstop
    diff_c = im_c / factor - topo_c - offset
    diff_b = diff_c * mask_bs
    sum_c = im_c / factor + topo_c - offset
    sum_b = sum_c * mask_bs

    if heraldo is True:
        #holo = sum_c * heraldo_operator * mask_bs*(1 - mask_pixel_smooth)
        holo = diff_c * heraldo_operator * mask_bs*(1 - mask_pixel_smooth)
    elif heraldo is False:
        #holo = sum_c * mask_bs*(1 - mask_pixel_smooth)
        holo = diff_c * mask_bs*(1 - mask_pixel_smooth)

    # Reconstruct
    recon = cci.reconstruct(
        fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
        * np.exp(1j * phase)
    )

    # worker dictionary
    worker_dict = {}
    worker_dict["center"] = center
    worker_dict["topo_c"] = topo_c
    worker_dict["im_c"] = im_c
    worker_dict["recon"] = recon
    worker_dict["factor"] = factor
    worker_dict["offset"] = offset
    worker_dict["shift"] = shift
    worker_dict["diff_c"] = diff_c
    worker_dict["sum_c"] = sum_c
    worker_dict["mask_bs"] = mask_bs
    worker_dict["mask_pixel"] = mask_pixel
    
    return worker_dict

In [ ]:
# Setup phase and propagation for cdi once
phase_cdi = 0
prop_dist_cdi = 0
dx = 0
dy = 0


def phase_retrieval(
    pos, neg, mask_pixel, supportmask, vmin = 0, Startimage=None, Startgamma=None
):
    # Prepare Input holograms
    pos2 = pos.copy()
    neg2 = neg.copy()

    mi, _ = np.percentile(pos2[pos2 != 0], [vmin, 99.9])
    pos2 = pos2 - mi
    mi, _ = np.percentile(neg2[neg2 != 0], [vmin, 99.9])
    neg2 = neg2 - mi

    pos2[pos2 < 0] = 0
    neg2[neg2 < 0] = 0
    pos2 = pos2.astype(complex)
    neg2 = neg2.astype(complex)

    bsmask_p = mask_pixel.copy()
    bsmask_p[pos2 <= 0] = 1
    bsmask_n = mask_pixel.copy()
    bsmask_n[neg2 <= 0] = 1

    # Setup start image and startgamma
    if Startimage is None:
        Startimage = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(supportmask)))
    else:
        Startimage = Startimage.copy()
    if Startgamma is None:
        Startgamma = np.ones(pos.shape) * 1e-6 * 2
        Startgamma[pos.shape[0] // 2, pos.shape[1] // 2] = 0.7
    else:
        Startgamma = Startgamma.copy()

    # Settings for phase retrieval reconstructions
    partial_coherence = True

    # Setup
    retrieved_p = np.zeros(pos2.shape, np.cdouble)
    retrieved_n = np.zeros(pos2.shape, np.cdouble)

    # Algorithms and Inital guess
    plt.rcParams["figure.dpi"] = 100
    print("CDI - larger mask")

    algorithm_list = ["mine", "mine", "mine"]
    Nit_list = [700, 50, 50]  # iterations for algorithm_list

    x = (np.sqrt(np.maximum(pos2, np.zeros(pos2.shape)))[mask_pixel == 0]).flatten()
    y = ((np.abs(Startimage))[mask_pixel == 0]).flatten()
    res = stats.linregress(x, y)
    Startimage -= res.intercept
    Startimage /= res.slope

    average_img = 30
    real_object = False  # always set to False

    if partial_coherence:
        RL_freq = 20
        RL_it = 50

        algorithm_list_pc = ["mine", "ER", "ER"]
        Nit_list_pc = [700, 50, 50]

    # Execute Phase retrieval
    start_time = time.time()
    for i in range(len(Nit_list) // 3):
        print("############ -   CDI")

        # Positive helicity - beta_mode="arctan"
        retrieved_p, Error_diff_p, Error_supp = PhR.PhaseRtrv_GPU(
            diffract=np.sqrt(np.maximum(pos2, np.zeros(pos2.shape))),
            mask=supportmask,
            mode=algorithm_list[3 * i],
            beta_zero=0.5,
            Nit=Nit_list[3 * i],
            beta_mode="arctan",
            plot_every=349,
            Phase=Startimage,
            seed=False,
            real_object=real_object,
            bsmask=bsmask_p,
            average_img=average_img,
            Fourier_last=True,
        )

        # Positive helicity - beta_mode="const"
        retrieved_p, Error_diff_p2, Error_supp = PhR.PhaseRtrv_GPU(
            diffract=np.sqrt(np.maximum(pos2, np.zeros(pos2.shape))),
            mask=supportmask,
            mode=algorithm_list[3 * i + 1],
            beta_zero=0.5,
            Nit=Nit_list[3 * i + 1],
            beta_mode="const",
            plot_every=24,
            Phase=retrieved_p,
            seed=False,
            real_object=real_object,
            bsmask=bsmask_p,
            average_img=average_img,
            Fourier_last=True,
        )

        # Negative helicity - beta_mode="arctan"
        retrieved_n, Error_diff_n2, Error_supp = PhR.PhaseRtrv_GPU(
            diffract=np.sqrt(np.maximum(neg2, np.zeros(neg2.shape))),
            mask=supportmask,
            mode=algorithm_list[3 * i + 2],
            beta_zero=0.5,
            Nit=Nit_list[3 * i + 2],
            beta_mode="const",
            plot_every=24,
            Phase=retrieved_p * np.sqrt(np.sum(neg2) / np.sum(pos2)),
            seed=False,
            real_object=real_object,
            bsmask=bsmask_n,
            average_img=average_img,
            Fourier_last=True,
        )

        print("--- %s seconds ---" % np.round((time.time() - start_time), 2))

        Startimage = retrieved_p.copy()

        # Partial coherence phase retrieval
        if partial_coherence:
            # CDI_PC
            print("############   -   CDI_pc")
            pos3 = (np.abs(retrieved_p) ** 2) * bsmask_p + np.maximum(
                pos2, np.zeros(pos2.shape)
            ) * (1 - bsmask_p)
            neg3 = (np.abs(retrieved_n) ** 2) * bsmask_n + np.maximum(
                neg2, np.zeros(neg2.shape)
            ) * (1 - bsmask_n)

            # retrieve pos image
            (
                retrieved_p_pc,
                Error_diff_p_pc,
                Error_supp,
                gamma_p,
            ) = PhR.PhaseRtrv_with_RL(
                diffract=np.sqrt(pos3),
                mask=supportmask,
                mode=algorithm_list_pc[3 * i],
                beta_zero=0.5,
                Nit=Nit_list_pc[3 * i],
                beta_mode="arctan",
                gamma=Startgamma,
                RL_freq=RL_freq,
                RL_it=RL_it,
                plot_every=349,
                Phase=Startimage,
                seed=False,
                real_object=False,
                bsmask=np.zeros(bsmask_p.shape),
                average_img=average_img,
                Fourier_last=True,
            )

            (
                retrieved_p_pc,
                Error_diff_p_pc2,
                Error_supp,
                gamma_p,
            ) = PhR.PhaseRtrv_with_RL(
                diffract=np.sqrt(pos3),
                mask=supportmask,
                mode=algorithm_list[3 * i + 1],
                beta_zero=0.5,
                Nit=Nit_list_pc[3 * i + 1],
                beta_mode="const",
                gamma=gamma_p,
                RL_freq=RL_freq,
                RL_it=RL_it,
                plot_every=24,
                Phase=retrieved_p_pc,
                real_object=False,
                bsmask=np.zeros(bsmask_p.shape),
                average_img=average_img,
                Fourier_last=True,
            )
            (
                retrieved_n_pc,
                Error_diff_n_pc2,
                Error_supp,
                gamma_n,
            ) = PhR.PhaseRtrv_with_RL(
                diffract=np.sqrt(neg3),
                mask=supportmask,
                mode=algorithm_list[3 * i + 2],
                beta_zero=0.5,
                Nit=Nit_list_pc[3 * i + 2],
                beta_mode="const",
                gamma=gamma_p,
                RL_freq=RL_freq,
                RL_it=RL_it,
                plot_every=24,
                Phase=retrieved_p_pc * np.sqrt(np.sum(neg2) / np.sum(pos2)),
                real_object=False,
                bsmask=np.zeros(bsmask_n.shape),
                average_img=average_img,
                Fourier_last=True,
            )

            print("--- %s seconds ---" % np.round((time.time() - start_time), 2))

            Startimage = retrieved_p_pc.copy()
            Startgamma = gamma_p.copy()

    print("Phase Retrieval Done!")

    return (
        retrieved_p,
        retrieved_n,
        retrieved_p_pc,
        retrieved_n_pc,
        bsmask_p,
        bsmask_n,
        gamma_p,
        gamma_n,
    )

In [ ]:
def optimize_phase_contrast(recon, supportmask, method="contrast", prefered_color=None):
    """
    Automatically shifts contrast of phase retrieval reconstruction into real part

    Parameter
    =========
    recon : complex array
        FTH/CDI reconstruction plane (Patterson map)
    supportmask : array
        Supportmask of Patterson map for phase retrieval
    method : string
        Choose method for phase optimization ("contrast","minima","maxima")
    prefered_color : string or None
        Shift contrast such that color of domains with largest are is white ("white"),
        black ("black") or non-specific (None)

    Output
    ======
    recon_optimized: complex array
        reconstruction with optimized contrast
    optimized_phase: complex scalar
        phase corresponding to optimized reconstruction
    ======
    author: ck 2023
    """

    # filter references from supportmask
    mask = supportmask.copy()
    mask = mask.astype(bool)
    mask = skimage.morphology.remove_small_objects(mask, min_size=200)
    mask = mask.astype(float)

    # Make object aperture smaller to minimize edge effects
    footprint = skimage.morphology.disk(4)
    mask = skimage.morphology.erosion(mask, footprint)

    # Gaussian filter to remove high intensity peaks
    reco = scipy.ndimage.gaussian_filter(recon, 1)

    # Different functions for optimization
    def contrast(phi, reco, tmask):
        temp = np.imag(reco * np.exp(1j * phi)) * tmask
        mi, ma = np.percentile(temp[temp != 0], [1, 99])
        contrast = ma - mi
        return contrast

    def minima(phi, reco, tmask):
        tmp = np.real(reco * np.exp(1j * phi)) * tmask
        minima, maxima = np.percentile(tmp[tmp != 0], [0.01, 99])
        return minima

    def maxima(phi, reco, tmask):
        tmp = np.real(reco * np.exp(1j * phi)) * tmask
        minima, maxima = np.percentile(tmp[tmp != 0], [0.01, 99])
        return maxima

    # Choose optimization method
    if method == "minima":
        optimized_phase = scipy.optimize.fminbound(
            minima, -np.pi, np.pi, args=(recon, mask), disp=False
        ).astype(float)
    elif method == "contrast":
        optimized_phase = scipy.optimize.fminbound(
            contrast, -np.pi, np.pi, args=(recon, mask), disp=False
        ).astype(float)
    elif method == "max":
        optimized_phase = scipy.optimize.fminbound(
            contrast, -np.pi, np.pi, args=(recon, mask), disp=False
        ).astype(float)

    # Calc optimized reconstruction
    recon_optimized = recon * np.exp(1j * optimized_phase)

    # Optional: Shift phase such that "background" are white or black
    mi_p, ma_p = np.percentile(np.real(recon_optimized[mask == 1]), [1, 99])
    mean = np.mean(np.real(recon_optimized[mask == 1]))
    if prefered_color == "white":
        # Make white domains the dominant domains
        if mean < (mi_p + ma_p) / 2:
            optimized_phase = optimized_phase + np.pi
            recon_optimized = recon * np.exp(1j * optimized_phase)
    elif prefered_color == "black":
        # Make black domains the dominant domains
        if mean > (mi_p + ma_p) / 2:
            optimized_phase = optimized_phase + np.pi
            recon_optimized = recon * np.exp(1j * optimized_phase)

    return recon_optimized, optimized_phase

In [ ]:
def correct_background(image,heraldo = True):
    # Inhomogen vertical background
    offset = gaussian_filter(np.mean(image[-10:],axis=0),2)
    mask_vert = np.ones(image.shape)
    mask_vert = offset * mask_vert
    image = image - mask_vert
    
    # Horizontal stripes
    if heraldo is False:
        offset = np.mean(image[:,-5:],axis=1)
        offset = gaussian_filter(offset,2)
        offset[:409] = 0
        offset[430:483] = 0
        offset[508:614] = 0
        offset[630:] = 0
        mask_hor = np.ones(image.shape)
        mask_hor = np.rot90(offset * mask_hor,-1)
        image = image - mask_hor
    
        # Inhomoge background
        background = mask_vert + mask_hor
    if heraldo is True:
        background = mask_vert.copy()
    
    return image, background

# Experimental Details

In [ ]:
# Dict with most basic experimental parameter
experimental_setup = {
    "ccd_dist": 0.20,  # ccd to sample distance
    "px_size": 13.5e-6,  # pixel_size of camera
    "binning": 1,  # Camera binning
}

# Setup for azimuthal integrator
detector = Detector(
    experimental_setup["binning"] * experimental_setup["px_size"],
    experimental_setup["binning"] * experimental_setup["px_size"],
)
experimental_setup["z_angle_offset"] = 67

# General saving folder and log folder
folder_general = sup.create_folder(join(BASEFOLDER, "processed"))
sup.create_folder(join(folder_general, "Logs"))
sup.create_folder(join(folder_general, "Topos"))
print("Output Folder: %s" % folder_general)

# Load images

Start by loading the images: image of interest (im), reference of charge scattering (topo), any kind of dark image (dark)

We estalished the following convention: Difference Hologram which contains only the magnetic scattering will be calculated according to:

$Diff = \frac{Image}{factor} - Topo$,

where the factor is used for intensity scaling. In Case that you recorded scans of the same magnetic state with both helicities, use the image with negative helicity as topo and the one with positive helicity as image

In [ ]:
# Define scan ids for each image
im_id = 609 # single helicity mode: image with magnetic contrast, double helicity: pos
dark_id = 608
topo_id = (
    607  # single helicity mode: image without magnetic contrast, double helicity: neg
)

# Are you using a HERALDO dataset?
heraldo = False

# Are you going to use ewalds sphere projection?
project_ewalds_sphere = False

# Load energy and add to experimental setup
experimental_setup["energy"] = load_pre_scan_snapshot(im_id,"energy")
experimental_setup["lambda"] = cci.photon_energy_wavelength(
    experimental_setup["energy"], input_unit="eV"
)

# Load angles
experimental_setup["srotx"] = load_pre_scan_snapshot(im_id,"srotx")
experimental_setup["srotz"] = load_pre_scan_snapshot(im_id,"srotz")
experimental_setup["angle"] = experimental_setup["srotz"] - experimental_setup["z_angle_offset"]

print("Image Id: %s" % im_id)
print("Topo Id: %s" % topo_id)
print("Dark Id: %s" % dark_id)
print("Sample rotation angle: %.2f"%experimental_setup["angle"])



## Load image of interest

In [ ]:
# Load image
image = load_processing(im_id, crop=None)

# Plot
fig, ax = cimshow(image)
ax.set_title("Image")

## Load topo data set and average

In [ ]:
# Load topo
topo = load_processing(topo_id , crop=None)

# Plot
fig, ax = cimshow(topo)
ax.set_title("Topo")

## Load dark image

In [ ]:
# Load image
if dark_id is not None:
    dark = load_processing(dark_id, crop=None)
    
    image = image - dark
    topo = topo - dark
    
    #image, background = correct_background(image,heraldo = heraldo)
    #topo, _ = correct_background(topo,heraldo = heraldo)
    
    # Plot
    fig, ax = cimshow(dark)
    ax.set_title("Dark")
    
    # Plot
    fig, ax = cimshow(background)
    ax.set_title("Background")

In [ ]:
# Plot
fig, ax = cimshow(image)
ax.set_title("Image")

## Create defect mask

In [ ]:
# Mask all detector crap. Will be shifted as according to center in processing pipeline
mask_defect = np.zeros(image.shape)
#mask_defect[499:500,80:370] = 1
#mask_defect[502:503,80:440] = 1
#mask_defect[505:506,130:380] = 1
#mask_defect[507:508,130:440] = 1
#mask_defect[499:500,730:] = 1
#mask_defect[198:199,887] = 1
#mask_defect[21:25,575] = 1
#mask_defect[212:213,183] = 1

# Center holograms

* Find center of the hologram to get a well-defined q-space. 
* Create smooth mask for beamstop or overexposed areas in direct beam

## Basic widget to find center

Try to **align** the circles to the **center of the scattering pattern**. Care! Position of beamstop might be misleading and not represent the actual center of the hologram. Circles are just a guide to eye and will not be used otherwise.

In [ ]:
# Find center position via widget
c0, c1 = [1014, 1103]  # initial values
#c0, c1 = [516, 509]  # initial values
#ic = interactive.InteractiveCenter(np.log10(image-np.min(image)+1), c0=c0, c1=c1)
ic = interactive.InteractiveCenter(image, c0=c0, c1=c1)

In [ ]:
# Get center positions
center = [ic.c0, ic.c1]
print(f"Center:", center)

## Azimuthal integrator widget for finetuning
More of an "expert widget" which works very well for alignment if you have an Airy Pattern as a scattering image. PyFai transforms images from carthesian detector coordinate system into polar coordinate system with angle `phi` and radial distance `q` as axis (azimuthal transformation). The center of the coordinate system will be defined in the azimuthal integrator class and must not necessarily represents the center coordinates of your image array. If the center is set correctly, all rings of the Airy pattern will be transformed into a straight line in the I(q,chi)-plot as rings appear at a given q for all angles chi.

In [ ]:
# Setup azimuthal integrator for virtual geometry
ai = AzimuthalIntegrator(
    dist=experimental_setup["ccd_dist"],
    detector=detector,
    wavelength=experimental_setup["lambda"],
    poni1=center[0]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # y (vertical)
    poni2=center[1]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # x (horizontal)
)

In [ ]:
# Not the widget, just for double checking to find correct radial range for plotting
# Perform azimuthal transformation
I_t, q_t, phi_t = ai.integrate2d(
    np.log10(image-np.min(image)+1),
    500,  # number of points for phi
    radial_range=(0.025, 0.1),  # relevant q-range
    unit="q_nm^-1",
    correctSolidAngle=False,
    method = "BBox"
)
# Combine in an xarray for plotting
az2d = xr.DataArray(I_t, dims=("phi", "q"), coords={"q": q_t, "phi": phi_t})

# Plot
fig, ax = plt.subplots()
mi, ma = np.percentile(I_t, [1, 95])
az2d.plot.imshow(ax=ax, vmin=mi, vmax=ma)
plt.title(f"Azimuthal integration")

In [ ]:
# The widget
aic = interactive.AzimuthalIntegrationCenter(
    #np.log10(image-np.min(image)+1),
    image,
    ai,
    c0=center[0],
    c1=center[1],
    im_data_range=[1, 98],
    radial_range=(0.025, 0.05),
    qlines=[100, 110],
)

In [ ]:
# Get center positions from widget
center = [aic.c0, aic.c1]
print(f"Center:", center)

## Centering of image hologram

In [ ]:
# Apply to topo and image
shift_c = np.array(image.shape) / 2 - center
im_c = cci.shift_image(image, shift_c)
topo_c = cci.shift_image(topo, shift_c)  # centered image
mask_defect_c = cci.shift_image(mask_defect, shift_c)
mask_defect_c[mask_defect>1e-3] = 1

# Image Registration

Relative drift between data holograms and their corresponding topo holograms is calculated by image registration algorithm. Necessary to get well defined difference hologram. The reference is always the static background image (topo).

## Set Alignment ROI 

Set a region of interest (ROI) of reference (topo) use for image registration is performed. Can include beamstop when beamstop mask was defined.

How to use:
1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
fig, ax = cimshow(im_c)
ax.set_title("Don't include the beamstop as this will misdirect the algorithm")

In [ ]:
# Takes start and end of x and y axis
x1, x2 = ax.get_xlim()
y2, y1 = ax.get_ylim()
roi_im_reg = np.array([y1, y2, x1, x2]).astype(int)
roi_im_reg = np.s_[roi_im_reg[0] : roi_im_reg[1], roi_im_reg[2] : roi_im_reg[3]]

print(f"Image registration roi:", roi_im_reg)

## Calculate drift of images

In [ ]:
shift = cci.image_registration(
    im_c[roi_im_reg],
    topo_c[roi_im_reg],
    method="dipy",
)
print("Relative shift is: %s"%shift)

In [ ]:
# Define shift manually for comparison
tmp_shift = [0.03, 0.02]

# Loop over shifts
temp_diff = np.zeros((3, im_c.shape[0], im_c.shape[1]))
shifts = [
    [0, 0],
    tmp_shift,
    -shift,
]
for i, tshift in enumerate(shifts):
    temp = cci.shift_image(im_c, shift)
    temp_factor = cci.dyn_factor(temp, topo_c)
    temp_diff[i] = temp - temp_factor[0] * topo_c

# Plots for comparision
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(12, 4))
mi, ma = np.percentile(temp_diff[0], [.1, 99.9])
ax[0].imshow(temp_diff[0], vmin=mi, vmax=ma)
ax[0].set_title("Zero shift")
mi, ma = np.percentile(temp_diff[1], [.1, 99.9])
ax[1].imshow(temp_diff[1], vmin=mi, vmax=ma)
ax[1].set_title("Manual shift: %s" % shifts[1])
mi, ma = np.percentile(temp_diff[2], [.1, 99.9])
ax[2].imshow(temp_diff[2], vmin=mi, vmax=ma)
ax[2].set_title("Auto shift: %s" % np.round(shifts[2], 2))

In [ ]:
# If the shift is very small you might want to set it to [0,0]
# as this prevents some artifacts
shift = [0, 0]

## Correct drift of topo

In [ ]:
# Correct relative drift
topo_c = cci.shift_image(topo_c, shift)  

# Plot original and shifted holos
mi, ma = np.percentile(np.real(im_c[im_c != 0]), (0.1, 99))
fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(8, 4))
ax[0].imshow(np.real(topo), cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Uncentered topo")
ax[1].imshow(np.real(topo_c), cmap="viridis", vmin=mi, vmax=ma)
ax[1].set_title("Centered topo with beamstop")

# Add circles with different radi r
tmp = np.array(image.shape) / 2
for r in np.arange(50, 200, 25):
    ax[0].add_artist(plt.Circle((tmp[1], tmp[0]), r, fill=None, ec="red"))
    ax[1].add_artist(plt.Circle((tmp[1], tmp[0]), r, fill=None, ec="red"))

# Calculate Projection

In [ ]:
if project_ewalds_sphere is True:
    %%time
    proj_im = PhR.inv_gnomonic2(im_c,alpha=experimental_setup["angle"], center=None, experimental_setup = experimental_setup, method='cubic' , mask=None)
    proj_topo = PhR.inv_gnomonic2(topo_c,alpha=experimental_setup["angle"], center=None, experimental_setup = experimental_setup, method='cubic' , mask=None)
    
    im_c = proj_im.copy()
    topo_c = proj_topo.copy()

# Create beamstops

We want to cover the beamstop with a smooth circle to cover its sharp edges as these would create ringing-like artifacts in the reconstruction plane. Make it only as large as necessary to keep as much information as possible.

## Circle beamstop

Set beamstop diameter and std for smoothing filter. Higher values mean stronger smoothing. If you have a very small beamstop you might need to reduce the smoothing value. Otherwise the sharp gradient of the real beamstop will still be visible

In [ ]:
bs = interactive.InteractiveBeamstop(
    im_c, im_c.shape[0] / 2, im_c.shape[1] / 2, rBS=55, stdBS=3
)

In [ ]:
# Take value from widget and create beamstop mask
bs_diam = bs.rBS
bs_smoothing = bs.stdBS
mask_bs = 1 - cci.circle_mask(
    topo.shape, np.array(topo.shape) / 2, bs.rBS, sigma=bs_smoothing
)

# Apply beamstop to image
im_b = im_c * mask_bs

# Plot image with beamstop and valid pixel mask
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
mi, ma = np.percentile(im_b, [0.1, 99.9])
ax[0].imshow(im_b, cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Masked image")

ax[1].imshow(mask_bs)
ax[1].set_title("Beamstop mask")

ax[2].imshow(1 - mask_bs)
ax[2].set_title("1 - Beamstop mask")
plt.tight_layout()

## Manual masking of beamstop wires

Just mask the beamstop wires, broken pixels, etc. 

In [ ]:
poly_mask = interactive.draw_polygon_mask(im_c)

In [ ]:
# Take poly coordinates and mask from widget
p_coord = poly_mask.get_vertice_coordinates()
mask_draw = poly_mask.full_mask.astype(int)

print("Copy these coordinates into the 'load_poly_coordinates()' function:")
print(p_coord)

# Plot image with beamstop and valid pixel mask
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
mi, ma = np.percentile(im_c * (1 - mask_draw), [0.1, 99.9])
ax[0].imshow(im_c * (1 - mask_draw), cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Image * (1-mask_draw)")

mi, ma = np.percentile(im_c * mask_draw, [0.1, 99.9])
ax[1].imshow(im_c * mask_draw, vmin=mi, vmax=ma)
ax[1].set_title("Image * mask_draw")

ax[2].imshow(1 - mask_draw)
ax[2].set_title("1 - mask_draw")
plt.tight_layout()

In [ ]:
def load_poly_coordinates():
    """
    Dictionary that stores polygon corner coordinates of all drawn masks
    Example: How to add masks with name "test":
    mask_coordinates["test"] = copy coordinates from above
    """

    # Setup dictonary
    mask_coordinates = dict()

    # Setup dictonary
    mask_coordinates = dict()

    # Mask #1
    mask_coordinates["bs_medium"] = [[(972.7, 1009.9), (870.9, 1012.6), (773.4, 1014.8), (378.4, 1015.3), (2.9, 1011.5), (-7.2, 1032.1), (0.0, 1046.6), (205.3, 1048.0), (434.6, 1050.2), (752.6, 1049.7), (898.8, 1047.8), (970.3, 1043.0), (984.2, 1057.4), (1007.7, 1079.3), (1030.3, 1754.1), (1040.2, 2054.3), (1087.8, 2050.9), (1063.1, 1639.6), (1055.7, 1368.1), (1043.9, 1075.3), (1065.3, 1057.8), (1074.7, 1044.4), (1385.6, 1028.0), (1705.6, 1001.4), (1918.0, 981.7), (1957.4, 976.3), (2059.8, 984.4), (2055.8, 948.5), (1964.8, 937.6), (1644.5, 972.9), (1472.9, 984.4), (1289.9, 997.8), (1149.4, 1002.9), (1083.5, 1002.9), (1061.5, 986.4), (1039.7, 974.6), (1027.1, 596.5), (1007.1, -4.7), (968.6, -2.1), (984.0, 388.9), (992.9, 627.0), (1001.2, 846.5), (1006.7, 969.8), (1000.8, 978.4), (982.9, 995.0)]]
    mask_coordinates["bs_tiny"] = [[(1031.1221816767775, 977.4340148145258), (1034.0037306716035, 977.2570775955452), (1037.7326765093844, 971.856949685674), (1031.4787790844405, 968.2530087967233), (1028.192832979809, 972.5989375157521), (1029.8583443983453, 977.0801403765646)], [(1030.2067999471637, 998.8865063528044), (1026.1788660124541, 1002.3844489803155), (1028.2988312412488, 1007.6843620523017), (1032.9627547445966, 1008.3203516209401), (1035.4007147577104, 1005.1404037777482), (1034.8707234505118, 999.2045011371235)]]
    mask_coordinates["heraldo_scattering"] =[[(1007.4095176567635, 1013.4754968408944), (505.98321271519785, 1007.5186079574952), (-9.653967890197777, 1008.958935277622), (-12.534622530451372, 1039.205809000285), (510.3041946755783, 1044.967118280792), (977.339251289929, 1037.36694135153), (1009.1928384852571, 1035.1033445662363), (1005.8944251655911, 1662.665456693553), (1005.894425165591, 2049.4415501857748), (1045.7096112603783, 2052.285492049688), (1050.1476536059047, 1562.4318399769054), (1034.333843804725, 1034.154304768693), (1674.2207631852384, 1046.9520431563033), (2060.9968566774596, 1045.5300722243464), (2056.7309438815896, 1011.4027698573858), (1563.523108986949, 1002.38588865213), (1038.7155483948377, 1015.1231826692141), (1051.031606239122, 307.41630724946305), (1052.436869004418, -8.767814942121845), (1004.6579349843562, -5.957289411530041), (1014.4584508254043, 609.8046580666255)]]
    mask_coordinates["heraldo_horizontal"] = [[(-2.83452521656136, 1009.8480623206065), (-1.3531185573504558, 1035.5257777469287), (1025.5310994087342, 1035.299536429632), (1025.5310994087342, 1012.5846343217315)], [(1026.5271876719696, 1012.4405061539736), (1025.3049095314007, 1035.663790824784), (2057.4234810885005, 1042.1688630652416), (2054.602839225649, 1014.4325514138689)]]
    mask_coordinates["heraldo_vertical"] = [[(1012.9099229799814, -3.0803204926090757), (1012.5607844759745, 1019.0017005033029), (1040.495199813563, 1017.6986075745117), (1043.8484288127154, -3.080320492609019)], [(1012.5746307729039, 1025.679196136222), (1016.2184841327175, 2052.1676549458025), (1041.0549883806939, 2052.1676549458025), (1037.8428828310311, 1026.2245540943109)]]
    mask_coordinates["heraldo_horizontal_cropped"] = [[(-1.2, 487.7), (-3.2, 514.3), (520.0, 517.6), (520.3, 494.7)], [(1026.3, 498.0), (518.6, 494.4), (515.5, 518.5), (1026.6, 528.0)]]
    mask_coordinates["bs_tiny_wires"] = [[(516.3, 494.5), (518.4, 491.2), (520.0, 488.8), (520.2, 485.5), (517.7, 484.5), (517.6, 474.2), (518.7, 472.9), (518.6, 465.9), (520.0, 464.4), (519.5, 460.4), (518.7, 454.3), (519.5, 451.0), (519.3, 447.9), (519.5, 416.5), (519.5, 430.7), (520.9, 357.3), (520.7, 358.2), (520.7, 313.8), (521.6, 277.9), (523.2, 233.0), (518.7, 230.1), (518.0, 271.0), (517.3, 323.4), (517.0, 365.5), (516.1, 411.2), (515.7, 434.6), (515.4, 452.2), (513.8, 456.1), (513.8, 459.7), (513.8, 465.1), (514.3, 474.4), (513.6, 484.1), (511.9, 486.1), (512.8, 492.2), (513.3, 493.7)], [(511.7, 535.8), (510.5, 550.3), (510.3, 572.6), (508.2, 624.8), (506.7, 674.0), (504.2, 753.9), (510.0, 754.3), (511.0, 690.4), (512.6, 636.1), (513.9, 591.4), (514.6, 571.7), (515.0, 558.5), (515.2, 535.1)]]
    return mask_coordinates

In [ ]:
# Which drawn masks do you want to load? You can combine multiple masks from
# load_poly_coordinates(). Just add names of mask as strings to list like
# ["bs_small","bs_medium"]
polygon_names = ["bs_medium"]#["heraldo_horizontal_cropped"]
mask_draw = load_poly_masks(
    im_c.shape,
    polygon_names,
)
#mask_draw = np.zeros(im_c.shape)

# Expand/shrink Mask if necessary
#footprint = skimage.morphology.disk(3)
#mask_draw = skimage.morphology.dilation(mask_draw, footprint) # increase size
#mask_draw = skimage.morphology.erosion(mask_draw, footprint) # decrease size

# The relative position of the drawn beamstop with respect to the actual beamstop
# might change due to sample change, sample drift, realignment, etc. The drawn
# beamstop mask would therefore not cover the actual beamstop position. This function
# aligns the beamstop mask with respect to the actual position
optimize_position = False

# Optimize position of drawn mask relative to target image
if optimize_position is True:
    # level 1 (rough)
    optimized_shift, mask_shifted, _ = auto_shift_mask(
        mask_draw,
        im_c,
        shift_range_y=[-10,10],
        shift_range_x=[-10, 10],
        step_size=4,
        crop=300,
    )
    ## Level 2 (fine)
    optimized_shift, mask_shifted, overlap = auto_shift_mask(
        mask_draw,
        im_c,
        shift_range_y=[optimized_shift[0] - 4, optimized_shift[0] + 4],
        shift_range_x=[optimized_shift[1] - 4, optimized_shift[1] + 4],
        step_size=0.5,
        crop=300,
    )
    mask_draw = mask_shifted.copy()

# Shift mask
#mask_draw = np.round(cci.shift_image(mask_draw,[2,0]))

# Plot image with beamstop and valid pixel mask
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
mi, ma = np.percentile(im_c * (1 - mask_draw), [0.1, 99.9])
ax[0].imshow(im_c * (1 - mask_draw), cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Image * (1-mask_draw)")

#mi, ma = np.percentile(im_c * mask_draw, [0.1, 90])
ax[1].imshow(im_c * mask_draw, vmin=mi, vmax=ma)
ax[1].set_title("Image * mask_draw")

ax[2].imshow(1 - mask_draw)
ax[2].set_title("1 - mask_draw")

## Finetuning of mask position

In [ ]:
# Use widget to shift and expand or shrink the mask
ss_mask = interactive.Shift_Scale_Mask(im_c,mask_draw)

In [ ]:
# Take mask, shift and scaling from widget
mask_draw, mask_shift, mask_scale = ss_mask.get_mask()

## Overview beamstops
Verify good beamstop alignment

In [ ]:
# Use widget to shift and expand or shrink the mask
circ = 1 -cci.circle_mask(topo.shape, [1024,1024], 50, sigma=None)
circ_widget = interactive.Shift_Scale_Mask(im_c,circ)

In [ ]:
# Take mask, shift and scaling from widget
mask_circ, _, _ = circ_widget.get_mask()

In [ ]:
# Add circular beamstop mask
mask_im = mask_draw.copy() #+ mask_defect_c
mask_im = mask_im + 1 - mask_circ
mask_topo = mask_im.copy()

# Mask over-saturated pixel
mask_im = mask_im + (im_c > 40000)
mask_topo = mask_topo + (topo_c > 40000)

# Combine both
mask_pixel = mask_im + mask_topo
mask_pixel[mask_pixel > 1] = 1

# Create smooth mask for FTH reconstructions
footprint = skimage.morphology.disk(6)
mask_pixel_smooth = skimage.morphology.dilation(mask_pixel, footprint)
mask_pixel_smooth = gaussian_filter(mask_pixel_smooth, 2)

# Plot both
fig, ax = plt.subplots(2, 4, figsize=(10, 5), sharex=True, sharey=True)
mi, ma = np.percentile(im_c, [1, 99.9])
ax[0, 0].imshow(im_c, vmin=mi, vmax=ma)
ax[0, 0].set_title("Image")
mi, ma = np.percentile(im_c * mask_im, [1, 99.99])
ax[0, 1].imshow(im_c * mask_im, vmin=mi, vmax=ma)
ax[0, 1].set_title("Image*mask")
mi, ma = np.percentile(im_c * (1 - mask_im), [0.1, 99.9])
ax[0, 2].imshow(im_c * (1 - mask_im), vmin=mi, vmax=ma)
ax[0, 2].set_title("Image*(1-mask)")
ax[0, 3].imshow(mask_pixel_smooth)
ax[0, 3].set_title("Combined Mask")

mi, ma = np.percentile(topo_c, [1, 99.9])
ax[1, 0].imshow(topo_c, vmin=mi, vmax=ma)
ax[1, 0].set_title("Topo")
mi, ma = np.percentile(topo_c * mask_im, [1, 99.99])
ax[1, 1].imshow(topo_c * mask_topo, vmin=mi, vmax=ma)
ax[1, 1].set_title("Topo*mask")
mi, ma = np.percentile(topo_c * (1 - mask_topo), [0.1, 99.9])
ax[1, 2].imshow(topo_c * (1 - mask_topo), vmin=mi, vmax=ma)
ax[1, 2].set_title("topo*(1-mask)")
mi, ma = np.percentile((im_c - topo_c) * (1 - mask_pixel_smooth), [0.1, 99.9])
ax[1, 3].imshow((im_c - topo_c) * (1 - mask_pixel_smooth), vmin=mi, vmax=ma)
ax[1, 3].set_title("Image-Topo")

# Calculate difference holograms

You can see the reconstrution of the magnetization only after subtracting the large background that you get from the diffraction on the circular object aperture (Airy Pattern). This might require a scaling factor to correct intensity changes between the hologram and the topo. Scaling factor will be determined automatically by a linear fit. If the fit seems off, there might be an issue with the data

In [ ]:
# Get scaling factor and offset
factor, offset = cci.dyn_factor(
    im_c * (1 - mask_pixel),
    topo_c * (1 - mask_pixel),
    method="correlation",
    print_out=True,
    plot=True,
)

# Calculate differences (magnetic) and sums (topographc) contrast holograms.
# _c: centered, without beamstop, _b: centered, with beamstop
diff_c = im_c / factor - topo_c - offset
diff_b = diff_c * mask_bs
sum_c = im_c / factor + topo_c - offset
sum_b = sum_c * mask_bs

In [ ]:
# Plot an example of the difference or sum hologram
fig, ax = cimshow(diff_c * (1 - mask_pixel_smooth))
ax.set_title(f" Diff Id %d - %d" % (im_id, topo_id))

# fig, ax = cimshow(sum_b)
# ax.set_title(f" Sum Id %d" % im_id)

# Reconstruct Diff Holos (FTH)

Reconstruct the hologram.

0. If you are doing heraldo, determine the rotation angle of the hologram
1. Choose a region of interest (ROI) which means selecting one reconstruction from the reconstruction plane.
2. Propagate the image and shift the phase for maximal contrast and sharpness in your ROI

In [ ]:
# This is only relevant if you are doing heraldo
if heraldo is True:
    #xticks, yticks = [1014,1036], [1014,1036]
    xticks, yticks = [495,522], [490,517]
    shift_rotate_widget = interactive.Shift_Rotate(im_c, shift = [0,0], angle = 0, ticks = (yticks,xticks))

## Set Patterson Map ROI

Choose the reconstructions as the ROI.

1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
# Choose contrast mode
# diff_c: magnetic contrast only
# sum_c: topographic contrast only
holo = diff_c * mask_bs*(1 - mask_pixel_smooth)
#holo = (pos-neg) * mask_bs*(1 - mask_pixel_smooth)
holo = sum_c * mask_bs*(1 - mask_pixel_smooth)
#holo = im_c * mask_bs*(1 - mask_pixel_smooth)
#crop  = 300
#holo = holo * cci.circle_mask(holo.shape,np.array(holo.shape)/2,300,10)
#holo = holo[crop:-crop:,crop:-crop:].copy() * cci.circle_mask(ho)

if heraldo is True:
    print("Doing Heraldo")
    _, _, heraldo_rotation = shift_rotate_widget.get_parameter()
    tmp, heraldo_operator  = reconstruct_heraldo(holo, experimental_setup, 
                              center = None, prop_dist = 0, phase = 0,angle = heraldo_rotation)
else:
    tmp = fth.reconstruct(holo)
    
fig, ax = cimshow(np.imag(tmp), cmap="gray")

In [ ]:
# Execute to get roi
x1, x2 = ax.get_xlim()
y2, y1 = ax.get_ylim()
roi = np.array([y1, y2, x1, x2]).astype(int)  # ystart, ystop, xstart, xstop
#roi = [612, 825, 705, 898]
#roi = [1305, 1818,  296 , 794]
#roi = [906, 997, 830, 919] # Co
#roi = [811, 990, 683, 868]# Dy
#roi = [311, 477, 175, 348]# Dy 2x2
#roi =[397, 485, 319, 406]# Co 2x2
#roi  = [438, 523, 618, 716]
roi_s = np.s_[roi[0] : roi[1], roi[2] : roi[3]]
print(f"Roi Reco:{roi}")

## Tune propagation and phase
Focus the image by tuning the propagation distance. This really works like focussing in a microscope.
Phase slider will move contrast between real and imaginary part. Usually we use the phase which maximizes the contrast in the real part.

In [ ]:
# Widget
if heraldo is True:
    #holo = sum_c * heraldo_operator * mask_bs*(1 - mask_pixel_smooth)
    holo = diff_c * heraldo_operator * mask_bs*(1 - mask_pixel_smooth)
elif heraldo is False:
    #holo = sum_c * mask_bs*(1 - mask_pixel_smooth)
    holo = diff_c * mask_bs*(1 - mask_pixel_smooth)

slider_prop, slider_phase, button = reco.propagate(
    holo,
    roi_s,
    phase = 0, #Initial value
    prop_dist = 0, #Initial value
    experimental_setup=experimental_setup,
    scale=(1, 99),
)

In [ ]:
# Read prop dist and phase from widget
prop_dist = slider_prop.value
phase = slider_phase.value

print(f"Propagation distance: %0.2f" % prop_dist)
print(f"Phase: %0.2f" % phase)

In [ ]:
# Ugly widget to change all parameters
temp_factor = factor
temp_shift = shift.copy()
temp_phase = phase

# Open figure
fig, [ax, ax2,ax3] = plt.subplots(1, 3, figsize=(12, 4))

# Widget
layout = ipywidgets.Layout(width="100%")
opts = dict(layout=layout)


@ipywidgets.interact(
    centery=(center[0] - 50, center[0] + 50, .25),
    centerx=(center[1] - 50, center[1] + 50, .25),
    tfactor=(
        0.6,
        3,
        0.0025,
    ),
    xshift=(-4, 4, 0.01),
    yshift=(-4, 4, 0.01),
    tphase=(-2 * np.pi, 2 * np.pi, 0.05),
    options=opts,
)
def update(
    centery=center[0],
    centerx=center[1],
    tfactor=factor,
    yshift=shift[0],
    xshift=shift[1],
    tphase=phase,
):  # initial values
    global temp_shift, temp_factor, temp_center, temp_diff, temp_sum, temp_phase  # ugly writing as global variable

    # Couple widget only and global variables
    temp_shift = [yshift, xshift]
    temp_factor = tfactor
    temp_center = [centery, centerx]
    temp_phase = tphase

    # Calc diff and sum holo
    shifted = cci.shift_image(im_c, temp_shift)
    temp_diff = shifted - temp_factor * topo_c
    temp_sum = shifted + temp_factor * topo_c

    temp_diff = cci.shift_image(
        temp_diff, -np.array([centery - center[0], centerx - center[1]])
    )
    temp_sum = cci.shift_image(
        temp_sum, -np.array([centery - center[0], centerx - center[1]])
    )

    if heraldo is True:
        temp_diff = temp_diff * heraldo_operator* (1 - mask_pixel_smooth)*mask_bs
        temp_sum = temp_sum * (1 - mask_pixel_smooth)*mask_bs*heraldo_operator
    else:
        temp_diff = temp_diff * (1 - mask_pixel_smooth)*mask_bs
        temp_sum = temp_sum * (1 - mask_pixel_smooth)*mask_bs

    # Reconstruction
    recon = cci.reconstruct(
        fth.propagate(
            temp_diff,
            prop_dist * 1e-6,
            experimental_setup=experimental_setup,
        )
        * np.exp(1j * temp_phase)
    )[roi_s]

    # Plots
    vmin, vmax = np.percentile(np.real(recon), (1, 99))
    ax.imshow(np.real(recon), vmin=vmin, vmax=vmax, cmap="gray")
    vmin, vmax = np.percentile(np.imag(recon), (1, 99))
    ax2.imshow(np.imag(recon), vmin=vmin, vmax=vmax, cmap="gray")
    vmin, vmax = np.percentile(np.abs(temp_diff * mask_bs * (1 - mask_pixel_smooth)), (0.1, 99.9))
    ax3.imshow(np.abs(temp_diff * (1-mask_pixel_smooth)), vmin=vmin, vmax=vmax)
    
    for r in np.arange(150, 300, 50):
        ax3.add_artist(plt.Circle(np.array(temp_diff.shape)/2, r, fill=None, ec="red"))

In [ ]:
# Copy values from widget
factor = temp_factor
shift = temp_shift
phase = temp_phase
center = temp_center.copy()
diff_c = np.abs(temp_diff.copy())
sum_c = np.abs(temp_sum.copy())
diff_b = diff_c * mask_bs
sum_b = sum_c * mask_bs

## Save reconstruction

Save png files of the images and a h5 file containing all important variables

In [ ]:
# Style of reconstruction plot
def plot_recon(recon, title):
    # Plot
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    fig.suptitle(title)

    vmin, vmax = np.percentile(np.real(recon), (1, 99))
    t_im1 = ax[0].imshow(np.real(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[0].set_title("Real")
    plt.colorbar(t_im1, ax=ax[0], aspect=50)

    vmin, vmax = np.percentile(np.imag(recon), (1, 99))
    t_im2 = ax[1].imshow(np.imag(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[1].set_title("Imag")
    plt.colorbar(t_im2, ax=ax[1], aspect=50)

In [ ]:
def get_title(data_key,im_id,topo_id,CDI=False):
    # Rotation in title
    #values = np.mean(np.array(load_pre_scan_snapshot(im_id, data_key)) * 1000)
    #values = [np.round(values, 2)]
    
    # Rotation in title
    values = load_pre_scan_snapshot(im_id, data_key)

    if CDI is False:
        mode = "FTH"
    elif CDI is True:
        mode = "CDI"
        
    if data_key == "srotz":
        title = "Image %d - %d %s: %s = %.1f (%.1f deg)" % (im_id, topo_id, mode, data_key, values, values-experimental_setup["z_angle_offset"])
    else:
        title = "Image %d - %d %s: %s = %.2f "%(im_id, topo_id, mode, data_key, values)
        
    return title

In [ ]:
# Create plot
holo = diff_c * mask_bs * (1 - mask_pixel_smooth)

# Reconstruct
if heraldo is True:
    recon, _ = reconstruct_heraldo(holo,experimental_setup,prop_dist = prop_dist, phase = phase, angle = heraldo_rotation)
elif heraldo is False:
    recon = fth.reconstruct(
        fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
        * np.exp(1j * phase)
    )

# Create plot
title = get_title("srotz",im_id,topo_id)
plot_recon(recon[roi_s], title)

# Save images
fname = join(
    folder_general,
    "Recon_ImId_%04d_RefId_%04d_%s.png" % (im_id, topo_id, USER),
)
print("Saving: %s" % fname)
plt.savefig(fname, bbox_inches="tight", transparent=False)

# Save hdf5 file
save_fth_h5()

In [ ]:
# Closes all existing plots
plt.close("all")

# Batch processing FTH

In [ ]:
# Define the sets for reconstructions. You can make a list or use np.arange
# im_id_set should always have ids of positive helicity holograms,
# topo_id_set those of negative helicity or a hologram of a saturated state

# You can also use nested lists: 
# in case topo_id_set = [[id1,id2],id3 it will use the sum hologram 
# calculated from [id1,id2] as topo

im_id_set = [1749]
dark_id_set = [1748]
topo_id_set = [1747]

# In case of single helicity reconstructions, adapt the helicity
# for contrast inversion
helicity = 1 * np.ones(len(im_id_set), dtype=int)  # [1,-1]

print("Dynamics Set:  %s" % im_id_set)
print("Reference Set: %s" % topo_id_set)
print("Helicity: %s" % helicity)

In [ ]:
# less ugly Automatic processing of image stacks
recons_name = []  # for gifs
for it, im_id in enumerate(im_id_set):
    # Load energy and add to experimental setup
    experimental_setup["energy"] = load_pre_scan_snapshot(im_id,"energy")
    experimental_setup["lambda"] = cci.photon_energy_wavelength(
        experimental_setup["energy"], input_unit="eV"
    )
    
    # Load angles
    experimental_setup["srotx"] = load_pre_scan_snapshot(im_id,"srotx")
    experimental_setup["srotz"] = load_pre_scan_snapshot(im_id,"srotz")
    experimental_setup["angle"] = experimental_setup["srotz"] - experimental_setup["z_angle_offset"]
    
    # Load images
    image = load_processing(im_id)

    # Load dark
    dark_id = dark_id_set[it]
    if dark_id is not None:
        dark = load_processing(dark_id, crop=None)
        image = image - dark

    # Get topo
    topo_id = topo_id_set[it]

    # Construct topo holo from two helicity images
    if isinstance(topo_id, list):
        print("Using Topo from sum of two helicity holograms")
        pos_id = topo_id[0]
        neg_id = topo_id[1]

        try:
            topo = load_topo_holo(pos_id, neg_id) / 2
        except:
            topo = load_topo_holo(neg_id, pos_id) / 2
        topo = cci.shift_image(
            topo, -shift_c
        )  # shift out of center so you don't need to change the worker

        # Process images
        worker_dict = worker(image, topo)

    # usual case, single topo holo
    else:
        # Load data
        print(f"Loading imageId: %04d, topoId: %04d" % (im_id, topo_id))
        topo = load_processing(topo_id)

        if dark_id is not None:
            topo = topo - dark

        # Process images
        worker_dict = worker(image, topo)

        # Save topo hologram
        save_topo_holo(worker_dict["sum_c"], im_id, topo_id)        

    # Reconstruct
    recon = worker_dict["recon"]
    
    # Plot
    title = get_title("magOOP", im_id, topo_id, CDI=False)
    plot_recon(recon[roi_s], title)

    # Save images
    fname = join(
        folder_general,
        "Recon_ImId_%04d_RefId_%s_fth_diff_stack_%s.png" % (im_id, topo_id, USER),
    )
    print("Saving: %s" % fname)
    plt.savefig(fname, bbox_inches="tight", transparent=False)

    # Optional: Save hdf5 file of fth data
    save_fth_h5()

    print(" ")
print("FTH stack processing finished")

# CDI Reconstruction

## Create set of pos and neg helicity holograms

CDI algorithm needs holograms recorded wih both helicity ($\sigma = \pm 1$) as input. We use will calculate those from our previously centered and intensity normalized holograms according to;´; 

$Image(\sigma) = Topo + \sigma \cdot diff $,

In [ ]:
# Copy values from FTH reco (here topo = sum_c)
pos = (sum_c + diff_c) / 2
neg = (sum_c - diff_c) / 2

## Create Support mask

### Option 1: Execute if you want to create a new support mask with elliptical mask widget

If you really want to create a new support mask, execute next cell and then the "InteractiveCircleCoordinates"-widget

In [ ]:
# How many references do you have?
nr_ref = 3

# Setup coordinates (nr_ref + 1 coordinates, as there is always the object aperture)
support_coordinates = [
    [(pos.shape[-2] // 2, pos.shape[-1] // 2), 10, 10, 0] for k in range(nr_ref + 1)
]

# Widget to find the positions and sizes of the different apertures
print("Cover the object & reference apertures for each set of reconstructions that originates from the same reference with circles.")
print("Optimization: Change one circle parameter, calc phase retrieval image, compare with images reconstructed with old circle parameter. Repeat!")

# Create plot
#holo = diff_c * mask_bs* (1 - mask_pixel_smooth)
holo = pos.copy()# * mask_bs* (1 - mask_pixel_smooth)

# Reconstruct
if heraldo is True:
    recon, _ = reconstruct_heraldo(holo,experimental_setup,prop_dist = prop_dist, phase = phase, angle = heraldo_rotation)
elif heraldo is False:
    recon = fth.reconstruct(
        fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
        * np.exp(1j * phase)
    )
recon = np.abs(recon)
    
ds_ellipse = interactive.InteractiveEllipseCoordinates(recon,
    len(support_coordinates),
    coordinates=support_coordinates,
)

In [ ]:
# Take coordinates of circles from widget
support_coordinates = ds_ellipse.get_params()

# Create supportmask from coordinates
supportmask = create_ellipse_supportmask(support_coordinates, pos.shape)

# Add heraldo slit
#supportmask[1018:1030,880:1024] = 1

# Plot supportmask as overlay
fig, ax = plt.subplots(figsize=(6, 6))
mi, ma = np.percentile(recon, (1, 99))
ax.imshow(recon, vmin=mi, vmax=ma, cmap="gray")
ax.imshow(supportmask, alpha=0.4, cmap="binary")
ax.set_title("Image with overlayed mask")

### Option 3: Load image under normal incidence, create mask and project mask

In [ ]:
# Define scan ids for image under normale incidence
load_id_im = 1024
load_id_topo = 1024

print("Reference CDI Image Id: %s" % load_id)

#### Load normal image and create support mask for that image

In [ ]:
# Load phase retrieval image and support mask
reference_dict = load_cdi(load_id_im,load_id_topo)
reference_cdi = reference_dict["reco_pc"] * reference_dict["supportmask"]
reference_cdi = np.abs(reference_cdi)

# Segment support mask into object and reference apertures
big_apertures = np.logical_not(skimage.morphology.remove_small_holes(np.logical_not(reference_dict["supportmask"]),area_threshold = 5000))
small_apertures = reference_dict["supportmask"] - big_apertures

# Improve support mask of big apertues
# Threshold filtering
supportmask = big_apertures*reference_cdi > 200000

# Remove artifacts from threshold filtering
footprint = skimage.morphology.disk(3)
supportmask= skimage.morphology.opening(supportmask, footprint)
supportmask = skimage.morphology.remove_small_holes(supportmask,area_threshold = 30)

# Smoothing of mask
supportmask = gaussian_filter(supportmask.astype(float),1.3)
supportmask = supportmask > 0.4

# Add references
supportmask = supportmask + small_apertures

# Plotting
fig, ax = plt.subplots(2,2,figsize=(12,12),sharex=True,sharey=True)
mi, ma = np.percentile(reference_cdi[reference_cdi!=0],[.1,99.9])
ax[0,0].imshow(reference_cdi,vmin=mi,vmax=ma,cmap="gray")
ax[0,0].set_xlim([700,1100])
ax[0,0].set_ylim([700,1100])
ax[0,0].invert_yaxis()
ax[0,1].imshow(supportmask)
ax[1,0].imshow(reference_cdi*supportmask,vmin=mi,vmax=ma,cmap="gray")
ax[1,1].imshow(reference_cdi*(1-supportmask),vmin=mi,vmax=ma,cmap="gray")

#### Project image onto new reconstruction of tilted sample

In [ ]:
# Plotting image
holo = pos * mask_bs* (1 - mask_pixel_smooth)

# Reconstruct
if heraldo is True:
    recon, _ = reconstruct_heraldo(holo,experimental_setup,prop_dist = prop_dist, phase = phase, angle = heraldo_rotation)
elif heraldo is False:
    recon = fth.reconstruct(
        fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
        * np.exp(1j * phase)
    )
recon = np.real(recon)

# projection
mask = get_mask_projections(supportmask, [np.deg2rad(experimental_setup["srotz"]-experimental_setup["z_angle_offset"])])

fig, ax = plt.subplots(2,2,figsize=(12,12),sharex=True,sharey=True)
mi, ma = np.percentile(recon[recon!=0],[.1,99.9])
ax[0,0].imshow(recon,vmin=mi,vmax=ma,cmap="gray")
ax[0,0].set_xlim([700,1100])
ax[0,0].set_ylim([700,1100])
ax[0,0].invert_yaxis()
ax[0,1].imshow(mask)
ax[1,0].imshow(recon*mask,vmin=mi,vmax=ma,cmap="gray")
ax[1,1].imshow(recon*(1-mask),vmin=mi,vmax=ma,cmap="gray")

In [ ]:
# Use widget to shift and expand or shrink the mask
ss_mask = interactive.Shift_Scale_Mask(recon,supportmask,cmap="gray")

In [ ]:
# Take mask, shift and scaling from widget
supportmask, mask_shift, mask_scale = ss_mask.get_mask()

### Option 4: Draw manually

In [ ]:
# Plotting image
holo = pos * mask_bs* (1 - mask_pixel_smooth)

# Reconstruct
if heraldo is True:
    recon, _ = reconstruct_heraldo(holo,experimental_setup,prop_dist = prop_dist, phase = phase, angle = heraldo_rotation)
elif heraldo is False:
    recon = fth.reconstruct(
        fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
        * np.exp(1j * phase)
    )
recon = np.real(recon)

poly_mask = interactive.draw_polygon_mask(recon,cmap='gray')

In [ ]:
# Take poly coordinates and mask from widget
p_coord = poly_mask.coordinates

# Polygon mask
manual_supportmask = poly_mask.full_mask.astype(int)

print("Copy these coordinates into the 'load_poly_coordinates()' function:")
print(p_coord)


In [ ]:
def load_poly_support():
    """
    Dictionary that stores polygon corner coordinates of all drawn masks
    Example: How to add masks with name "test":
    mask_coordinates["test"] = copy coordinates from above
    """

    # Setup dictonary
    mask_coordinates = dict()

    # Masks
    mask_coordinates["drawn_mask_heraldo_0deg"] = [[(964.5627505848117, 790.6253886706052), (958.9502425001773, 794.4839879787913), (957.0209428460842, 798.3425872869776), (950.005307740291, 800.4472778187155), (941.5865456133392, 804.1304862492569), (939.4818550816012, 806.4105676586397), (938.0787280604427, 813.4262027644329), (935.2724740181254, 815.5308932961708), (933.8693469969668, 818.512538216133), (931.2384838322943, 821.494183136095), (928.4322297899771, 825.8789550772157), (925.450584870015, 831.8422449171399), (926.6783210135287, 837.805534757064), (922.4689399500529, 844.2949972299227), (927.3798845241081, 853.5907137450987), (927.2044936464632, 866.7450295684608), (931.0630929546495, 871.4805832648711), (940.5342003474702, 882.5302085564954), (943.3404543897875, 889.0196710293541), (948.7775715967772, 889.0196710293541), (961.0549330319152, 896.5614787680817), (970.1752586694463, 897.6138240339507), (978.5940207963981, 900.4200780762679), (984.5573106363223, 896.210697012792), (995.9577176832362, 895.8599152575024), (1001.7456166455155, 893.22905209283), (1010.6905514054017, 892.176706826961), (1016.1276686123914, 886.5641987423264), (1016.3030594900363, 881.8286450459161), (1028.5804209251744, 872.1821467754505), (1027.5280756593054, 865.3419025473022), (1031.912847600426, 855.6954042768366), (1032.965192866295, 846.5750786393055), (1028.931202680464, 825.8789550772157), (1023.6694763511191, 819.3894926043571), (1020.3370496758673, 816.9340203173294), (1011.5675057936259, 805.8843950257052), (1003.1487436666741, 798.5179781646224), (995.7823268055913, 795.88711499995), (993.3268545185637, 791.5023430588293), (984.7327015139671, 789.5730434047362), (981.9264474716498, 791.5023430588293), (973.6830762223428, 791.677733936474)], [(724.7618360616121, 936.5561838840708), (721.176462855007, 935.1476444100474), (717.7191386914949, 935.9159386686057), (715.7984030450992, 938.4769195304665), (715.6703540020062, 942.7025379525369), (718.4874329500531, 945.6476659436768), (723.3532965875886, 945.9037640298628), (726.0423264925425, 944.6232735989324), (728.2191602251243, 942.0622927370716), (727.578915009659, 938.4769195304665)], [(1023.3663313017321, 1020.2409014928181), (877.1088756413031, 1023.0004761279206), (877.3848331048134, 1032.1070724237586), (1023.3663313017321, 1029.0715403251459)]]
    mask_coordinates["drawn_mask_heraldo_-35deg"] = [[(983.4763791077349, 782.8878502755655), (973.1682003936147, 785.9421254501196), (962.4782372826752, 790.5235382119508), (958.6603933144825, 793.9595977833243), (954.0789805526513, 798.1592261483363), (948.7339989971815, 803.5042077038061), (943.3890174417118, 808.0856204656372), (933.4626231244107, 823.356996338408), (930.0265635530372, 833.6651750525282), (930.0265635530372, 842.4462161793715), (931.5537011403144, 851.6090417030339), (934.6079763148686, 861.1536516235155), (938.8076046798805, 870.6982615439973), (943.3890174417118, 879.4793026708405), (951.0247053780971, 887.1149906072259), (959.8057465049403, 892.8417565595149), (969.350356425422, 899.3320913054425), (976.9860443618074, 902.3863664799967), (1067.0871620111548, 902.768150876816), (1079.6860471061907, 900.8592288927196), (1089.994225820311, 897.8049537181654), (1097.6299137566964, 893.2235409563342), (1101.0659733280697, 888.2603437976837), (1105.2656016930819, 885.5878530199489), (1110.6105832485514, 874.51610551219), (1112.1377208358285, 868.4075551630817), (1117.100917994479, 856.5722388616844), (1119.773408772214, 845.5004913539256), (1119.0098399785754, 835.9558814334439), (1114.4284272167442, 824.1205651320465), (1109.4652300580938, 813.8123864179263), (1101.0659733280697, 800.0681481324326), (1089.2306570266724, 790.9053226087701), (1080.8314002966486, 786.7056942437582), (1070.141437185709, 782.5060658787462)]]
    return mask_coordinates

In [ ]:
# Which drawn masks do you want to load? You can combine multiple masks from
# load_poly_coordinates(). Just add names of mask as strings to list like
# ["bs_small","bs_medium"]
coord = load_poly_support()
polygon_names = "drawn_mask_heraldo_0deg"
supportmask = create_polygon_mask(
    im_c.shape,
    coord[polygon_names],
)

# More preprocessing
footprint = skimage.morphology.disk(1)
supportmask= skimage.morphology.dilation(supportmask, footprint) # expand
supportmask= skimage.morphology.erosion(supportmask, footprint) # shrink

# Smoothing of mask
supportmask = gaussian_filter(supportmask.astype(float),2)
supportmask = supportmask > 0.4

# Plotting
fig, ax = plt.subplots(2,2,figsize=(12,12),sharex=True,sharey=True)
mi, ma = np.percentile(recon[recon!=0],[1,99])
ax[0,0].imshow(recon,vmin=mi,vmax=ma,cmap="gray")
ax[0,0].set_xlim([700,1100])
ax[0,0].set_ylim([700,1100])
ax[0,0].invert_yaxis()
ax[0,1].imshow(supportmask)

ax[1,0].imshow(recon*supportmask,vmin=mi,vmax=ma,cmap="gray")
ax[1,1].imshow(recon*(1-supportmask),vmin=mi,vmax=ma,cmap="gray")


### Option 2: Execute if you want to load an existing support mask created with elliptical mask widget

In [ ]:
def get_supportmask_coordinates(sample):
    """
    Dictionary that stores coordinates of circular support mask apertures
    """

    # Setup dictonary
    support_coord = dict()

    # coordinates
    support_coord["SWA-B31"] = [((1336.0, 935.0), 108.0, 108.0, 0.0), ((1445.6, 723.6), 25.0, 25.0, 0.0), ((1022.5, 601.0), 34.0, 34.0, 0.0), ((1024.0, 1024.0), 20.0, 20.0, 0.0)]
    support_coord["FBEb21_Co"] = [((873.5, 952.5), 84.5, 84.5, 0.0), ((828.1, 1022.0), 10.0, 10.0, 0.0), ((848.4, 1040.0), 13.0, 13.0, 0.0), ((886.0, 1034.5), 10.0, 10.0, 0.0), ((911.0, 1028.0), 12.0, 12.0, 0.0), ((1015.1, 864.8), 10.0, 10.0, 0.0), ((1024.0, 1024.0), 10.0, 10.0, 0.0)]
    support_coord["FBEb21_Co_2x2"] = [((361.7, 440.5), 86.0, 86.0, 0.0), ((316.2, 509.5), 12.0, 12.0, 0.0), ((337.1, 527.4), 14.0, 14.0, 0.0), ((373.9, 522.5), 11.0, 11.0, 0.0), ((398.4, 515.8), 14.0, 14.0, 0.0), ((503.2, 352.5), 11.0, 11.0, 0.0), ((512.0, 512.0), 12.0, 12.0, 0.0)]
    support_coord["FBEb21_Dy_2x2"] = [((269.0, 389.5), 139.5, 139.5, 0.0), ((185.9, 507.7), 20.0, 20.0, 0.0), ((221.9, 538.0), 15.0, 15.0, 0.0), ((286.5, 530.2), 15.0, 15.0, 0.0), ((325.0, 518.5), 16.0, 16.0, 0.0), ((497.5, 247.0), 15.0, 15.0, 0.0), ((512.0, 512.0), 15.0, 15.0, 0.0)]
    support_coord["SWA-B34"] = [((660.5, 717.5), 63.0, 63.0, 0.0), ((607.5, 630.5), 17.0, 17.0, 0.0), ((512.0, 512.0), 20.0, 20.0, 0.0)]
    support_coord["SWA-B31"] = [((667.0, 475.0), 97.5, 96.0, 0.0), ((720.5, 587.0), 23.5, 22.0, 0.0), ((731.5, 377.0), 23.0, 21.0, 0.0), ((512.0, 512.0), 11.5, 10.5, 0.0)]
    support_coord["SWA-B34-heraldo"] = [((488.5, 642.5), 53.0, 53.0, 0.0), ((435.0, 556.0), 9.0, 9.0, 0.0), ((340.5, 436.7), 15.0, 16.5, 36.0)]#,((340.5, 436.7), 15.0, 16.5, 36.0),((340.5, 436.7), 15.0, 16.5, 36.0)]
    return support_coord[sample]

In [ ]:
# Which supportmask to load? ("s2306a-C1", "s2308a-B1", ...)
sample = "SWA-B34-heraldo"

# Get coordinates and create supportmask
support_coordinates = get_supportmask_coordinates(sample)

In [ ]:
# Zeropadding to fix reco that are not in patterson map
pad_px = 100
pos = np.pad(pos,[[pad_px,pad_px],[pad_px,pad_px]])
neg = np.pad(neg,[[pad_px,pad_px],[pad_px,pad_px]])

cimshow(pos)

In [ ]:
# Widget to find the positions and sizes of the different apertures
print("Cover the object & reference apertures for each set of reconstructions that originates from the same reference with circles.")
print("Optimization: Change one circle parameter, calc phase retrieval image, compare with images reconstructed with old circle parameter. Repeat!")

# Plotting image
holo = sum_c * mask_bs* (1 - mask_pixel_smooth)

# Reconstruct
if heraldo is True:
    recon, _ = reconstruct_heraldo(holo,experimental_setup,prop_dist = prop_dist, phase = phase, angle = heraldo_rotation)
elif heraldo is False:
    recon = fth.reconstruct(
        fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
        * np.exp(1j * phase)
    )
recon = np.abs(recon)

ds_ellipse = interactive.InteractiveEllipseCoordinates(recon,
    len(support_coordinates),
    coordinates=support_coordinates,
)

In [ ]:
# Take coordinates of circles from widget
support_coordinates = ds_ellipse.get_params()

# Create supportmask from coordinates
supportmask = create_ellipse_supportmask(support_coordinates, pos.shape)

# Plotting image
holo = pos * mask_bs* (1 - mask_pixel_smooth)

# Reconstruct
if heraldo is True:
    # heraldo slit
    supportmask[508:516,512:582] = 1 #horz
    supportmask[554:625,363:369] = 1 #vert
    #supportmask[554:625,363:369] = 1 #vert
    
    # Do reco
    recon, _ = reconstruct_heraldo(holo,experimental_setup,prop_dist = prop_dist, phase = phase, angle = heraldo_rotation)
elif heraldo is False:
    recon = fth.reconstruct(
        fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
        * np.exp(1j * phase)
    )
recon = np.real(recon)


# Plot supportmask as overlay
fig, ax = plt.subplots(figsize=(6, 6))
mi, ma = np.percentile(recon, (1, 99))
ax.imshow(recon, vmin=mi, vmax=ma, cmap="gray")
ax.imshow(supportmask, alpha=0.4, cmap="binary")
ax.set_title("Image with overlayed mask")

### Take Roi
Choose the reconstructions as the ROI.

1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
fig, ax = cimshow(supportmask.astype(int))

## Do Phase Retrieval

In [ ]:
test_mask_pixel = np.zeros(pos.shape)
test_mask_pixel = mask_pixel.copy()
test_mask_pixel[995:] = 1
#test_mask_pixel = 1-mask_circ

In [ ]:
cimshow((1-test_mask_pixel)*pos)

In [ ]:
# Executes the algorithm
offset_vmin = .5
recon = fth.reconstruct(
    fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
    * np.exp(1j * phase)
)
Startimage = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(supportmask*recon)))
Startimage = None
Startgamma = None

(
    retrieved_p,
    retrieved_n,
    retrieved_p_pc,
    retrieved_n_pc,
    bsmask_p,
    bsmask_n,
    gamma_p,
    gamma_n,
) = phase_retrieval(pos, neg, test_mask_pixel.astype(int), supportmask, vmin = offset_vmin, Startimage=Startimage, Startgamma=Startgamma)

## Reconstruct images from phase retrieval

In [ ]:
# New beamstop for CDI recos as phase retrieval of low-q might be insufficient. If phase retrieval worked well
# Try without beamstop: `use_bs = False`
use_bs = False
bs_diam_cdi = 25  # diameter of beamstop

# Create beamstop
if use_bs is True:
    mask_bs_cdi = 1 - cci.circle_mask(
        topo.shape, np.array(topo.shape) / 2, bs_diam_cdi, sigma=4
    )
    mask_bs_cdi = 1 - mask_pixel_smooth.copy()
elif use_bs is False:
    mask_bs_cdi = np.ones(pos.shape)  # if you don't want a beamstop

# Get Recos
# Positive helicity
p = cci.reconstruct(
    fth.propagate(
        retrieved_p * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)

# Negative helicity
n = cci.reconstruct(
    fth.propagate(
        retrieved_n * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)


# Get Recos partial coherence
# Positiv partial coherence
p_pc = cci.reconstruct(
    fth.propagate(
        retrieved_p_pc * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)

# Negative partial coherence
n_pc = cci.reconstruct(
    fth.propagate(
        retrieved_n_pc * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)

# optimize phase
recon = p_pc - n_pc
#_, phase_cdi = optimize_phase_contrast(recon, supportmask, method="contrast",prefered_color="white")

# Plotting
mode = "-"
print("Fine-tuning of reconstruction parameter:")
slider_prop, slider_phase, slider_dx, slider_dy = rec.focusCDI(
    retrieved_p_pc * mask_bs_cdi,
    retrieved_n_pc * mask_bs_cdi,
    #retrieved_p * mask_bs_cdi,
    #retrieved_n * mask_bs_cdi,
    roi_cdi,
    mask=supportmask,
    phase=phase_cdi,
    dx=dx,
    dy=dy,
    prop_dist=prop_dist_cdi,
    experimental_setup=experimental_setup,
    operation=mode,
    max_prop_dist=20,
    scale=(1, 99),
)

In [ ]:
# Get phase from slider
phase_cdi = slider_phase.value
prop_dist_cdi = slider_prop.value

# Reconstruct images with new parameter
p_pc = fth.reconstructCDI(
    fth.propagate(
        retrieved_p_pc * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
) * np.exp(1j * phase_cdi)

n_pc = fth.reconstructCDI(
    fth.propagate(
        retrieved_n_pc * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
) * np.exp(1j * phase_cdi)


print("Phase CDI: %s" % phase_cdi)
print("Prop_dist: %s" % prop_dist_cdi)

In [ ]:
cimshow(np.real(p_pc+n_pc),cmap="gray")

In [ ]:
# Confirm that offset subtraction in cdi function works, i.e., only small fraction of hologram is actually masked
fig, ax = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)
tmp = np.abs(retrieved_p_pc*mask_bs_cdi)
mi, ma = np.percentile(tmp, [0.1, 99.9])
ax[0, 0].imshow(tmp, vmin=mi, vmax=ma)
ax[0, 0].set_title("Pos holo")

tmp = np.abs(retrieved_n_pc)
mi, ma = np.percentile(tmp, [0.1, 99.9])
ax[0, 1].imshow(tmp, vmin=mi, vmax=ma)
ax[0, 1].set_title("Neg holo")
ax[1, 0].imshow(bsmask_p)
ax[1, 0].set_title("Pos holo mask")
ax[1, 1].imshow(bsmask_n)
ax[1, 1].set_title("Neg holo mask")

In [ ]:
# Ta-ke initial values
# Open figure
fig, [ax, ax2,ax3] = plt.subplots(1, 3, figsize=(12, 4))

# Widget
layout = ipywidgets.Layout(width="100%")
opts = dict(layout=layout)


@ipywidgets.interact(
    centery=(center[0] - 10, center[0] + 10, .25),
    centerx=(center[1] - 10, center[1] + 10, .25),
    options=opts,
)
def update(
    centery=center[0],
    centerx=center[1],
    tphase=phase,
):  # initial values
    global temp_center, temp_phase  # ugly writing as global variable

    # Couple widget only and global variables
    temp_center = [centery, centerx]
    temp_phase = tphase

    temp_diff = cci.shift_image(
        retrieved_p_pc-retrieved_n_pc, -np.array([centery - center[0], centerx - center[1]])
    )
    temp_sum = cci.shift_image(
        retrieved_p_pc+retrieved_n_pc, -np.array([centery - center[0], centerx - center[1]])
    )

    temp_diff_b = temp_diff #* mask_bs * (1 - mask_pixel_smooth)

    # Reconstruction
    recon = cci.reconstruct(
        fth.propagate(
            temp_diff_b,
            prop_dist * 1e-6,
            experimental_setup=experimental_setup,
        )
        * np.exp(1j * temp_phase)
    )[roi_s]

    # Plots
    vmin, vmax = np.percentile(np.real(recon), (1, 99))
    ax.imshow(np.real(recon), vmin=vmin, vmax=vmax, cmap="gray")
    vmin, vmax = np.percentile(np.imag(recon), (1, 99))
    ax2.imshow(np.imag(recon), vmin=vmin, vmax=vmax, cmap="gray")

    
    # Add circles with different radi r
    vmin, vmax = np.percentile(temp_diff*mask_bs, (0.1, 99.9))
    ax3.imshow(np.abs(temp_diff*mask_bs), vmin=vmin, vmax=vmax)
    ax3.set_xlim([600,1400])
    ax3.set_ylim([600,1400])
    for r in np.arange(175, 450, 25):
        ax3.add_artist(plt.Circle(np.array(image.shape)/2, r, fill=None, ec="red"))

## Save reconstructions

In [ ]:
def plot_recon(recon, title):
    # Plot
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    fig.suptitle(title)

    vmin, vmax = np.percentile(np.real(recon), (.1, 99.9))
    t_im1 = ax[0].imshow(np.real(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[0].set_title("Real")
    plt.colorbar(t_im1, ax=ax[0], aspect=50)

    vmin, vmax = np.percentile(np.imag(recon), (.1, 99.9))
    t_im2 = ax[1].imshow(np.imag(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[1].set_title("Imag")
    plt.colorbar(t_im2, ax=ax[1], aspect=50)

In [ ]:
# Saves only real and imaginary part
#recon = p - n
recon = (p_pc - n_pc)#   / (p_pc + n_pc)

# Plot
title = get_title("srotz", im_id,topo_id, CDI=True)
#plot_recon(np.rot90(recon,-2)[roi_cdi], title)
plot_recon(recon[roi_cdi], title)

# Save images
fname = join(
    folder_general,
    "Recon_ImId_%04d_RefId_%s_cdi_diff_%s.png" % (im_id, topo_id, USER),
)
print("Saving: %s" % fname)
plt.savefig(fname, bbox_inches="tight", transparent=False)

# Save h5
save_cdi_h5()

In [ ]:
cimshow(neg)

# Batch processing CDI (needs to be updated)

## Define Scan Ids

In [ ]:
# Load support mask of which sample?
sample = "SWA-B31"

In [ ]:
# Define the sets for reconstructions. You can make a list or use np.arange
# im_id_set should always have ids of positive helicity holograms,
# topo_id_set those of negative helicity or a hologram of a saturated state

# You can also use nested lists: 
# in case topo_id_set = [[id1,id2],id3 it will use the sum hologram 
# calculated from [id1,id2] as topo

im_id_set = [1755,1763,1774]
dark_id_set = [1756,1762,1775]
topo_id_set = [1757,1761,1776]

# In case of single helicity reconstructions, adapt the helicity
# for contrast inversion
helicity = 1 * np.ones(len(im_id_set), dtype=int)  # [1,-1]

print("Dynamics Set:  %s" % im_id_set)
print("Reference Set: %s" % topo_id_set)
print("Helicity: %s" % helicity)

## Execute Phase Retrieval

In [ ]:
# Ugly Automatic processing of image stacks
recons_name = []  # for gifs
for it, im_id in enumerate(tqdm(im_id_set)):
    # Load energy and add to experimental setup
    experimental_setup["energy"] = load_pre_scan_snapshot(im_id,"energy")
    experimental_setup["lambda"] = cci.photon_energy_wavelength(
        experimental_setup["energy"], input_unit="eV"
    )
    
    # Load angles
    experimental_setup["srotx"] = load_pre_scan_snapshot(im_id,"srotx")
    experimental_setup["srotz"] = load_pre_scan_snapshot(im_id,"srotz")
    experimental_setup["angle"] = experimental_setup["srotz"] - experimental_setup["z_angle_offset"]
    
    # Load images
    image = load_processing(im_id)

    # Load dark
    dark_id = dark_id_set[it]
    if dark_id is not None:
        dark = load_processing(dark_id, crop=None)
        image = image - dark

    # Get topo
    topo_id = topo_id_set[it]

    # Construct topo holo from two helicity images
    if isinstance(topo_id, list):
        print("Using Topo from sum of two helicity holograms")
        pos_id = topo_id[0]
        neg_id = topo_id[1]

        try:
            topo = load_topo_holo(pos_id, neg_id) / 2
        except:
            topo = load_topo_holo(neg_id, pos_id) / 2
        topo = cci.shift_image(
            topo, -shift_c
        )  # shift out of center so you don't need to change the worker

        # Process images
        worker_dict = worker(image, topo)

    # usual case, single topo holo
    else:
        # Load data
        print(f"Loading imageId: %04d, topoId: %04d" % (im_id, topo_id))
        topo = load_processing(topo_id)

        if dark_id is not None:
            topo = topo - dark

        # Process images
        worker_dict = worker(image, topo)

        # Save topo hologram
        save_topo_holo(worker_dict["sum_c"], im_id, topo_id)   

    
    # Reconstruct
    recon = worker_dict["recon"]
    
    # Plot
    title = get_title("magOOP", im_id, topo_id, CDI=False)
    plot_recon(recon[roi_cdi], title)

    # Save images
    fname = join(
        folder_general,
        "Recon_ImId_%04d_RefId_%s_fth_diff_stack_%s.png" % (im_id, topo_id, USER),
    )
    print("Saving: %s" % fname)
    plt.savefig(fname, bbox_inches="tight", transparent=False)

    # Optional: Save hdf5 file of fth data
    save_fth_h5()

    ################ CDI ###############
    # Create pos and neg helicity set
    pos = (worker_dict["sum_c"] + worker_dict["diff_c"])/2
    neg = (worker_dict["sum_c"] - worker_dict["diff_c"])/2

    # Create mask pixel
    mask_im = worker_dict["mask_pixel"]
    mask_topo = mask_im.copy()

    # Mask over-saturated pixel
    mask_im = mask_im + (pos > 40000)
    mask_topo = mask_topo + (neg > 40000)

    # Combine both
    mask_pixel = mask_im + mask_topo
    mask_pixel[mask_pixel > 1] = 1

    # Create supportmask from coordinates
    supportmask = create_ellipse_supportmask(support_coordinates, pos.shape)

    # Do phase retrieval
    (
        retrieved_p,
        retrieved_n,
        retrieved_p_pc,
        retrieved_n_pc,
        bsmask_p,
        bsmask_n,
        gamma_p,
        gamma_n,
    ) = phase_retrieval(pos, neg, mask_pixel, supportmask, vmin = offset_vmin, Startimage=Startimage, Startgamma=Startgamma)

    # Get Recos partial coherence
    # Positiv partial coherence
    p_pc = fth.reconstructCDI(
        fth.propagate(
            retrieved_p_pc * mask_bs_cdi,
            prop_dist_cdi * 1e-6,
            experimental_setup=experimental_setup,
        )
    )
    # Negative partial coherence
    n_pc = fth.reconstructCDI(
        fth.propagate(
            retrieved_n_pc * mask_bs_cdi,
            prop_dist_cdi * 1e-6,
            experimental_setup=experimental_setup,
        )
    )

    ##### Calc reco and optimze contrast
    recon = helicity[it] * (p_pc - n_pc)
    _, phase_cdi = optimize_phase_contrast(recon, supportmask, method="contrast")
    # phase_cdi = 0
    recon = recon * np.exp(1j * phase_cdi)
    print("Phase is:", np.round(phase_cdi, 2))
    ########

    # Plot
    title = get_title("srotz",im_id,topo_id,CDI=True)
    plot_recon(recon[roi_cdi], title)

    # Save images
    fname = join(
        folder_general,
        "Recon_ImId_%04d_RefId_%s_cdi_stack_%s.png" % (im_id, topo_id, USER),
    )

    print("Saving: %s" % fname)
    plt.savefig(fname, bbox_inches="tight", transparent=False)
    recons_name.append(fname)

    # Save files as h5
    save_cdi_h5()

    print(" ")
print("CDI stack processing finished")

# Recos not in patterson map

In [ ]:
def pad_with_FFT(array,pad_px):
    padded_array = cci.FFT(np.pad(cci.reconstruct(array),[[pad_px,pad_px],[pad_px,pad_px]])).real
    return padded_array

In [ ]:
# Copy values from FTH reco (here topo = sum_c)
pos = (sum_c + diff_c) / 2
neg = (sum_c - diff_c) / 2

pad_px = 200

pos_pad = pad_with_FFT(pos,pad_px)
neg_pad = pad_with_FFT(neg,pad_px)
mask_bs_pad = pad_with_FFT(mask_bs,pad_px)
mask_pixel_pad = np.round(pad_with_FFT(mask_pixel,pad_px))
mask_pixel_smooth_pad = pad_with_FFT(mask_pixel_smooth,pad_px)

In [ ]:
cimshow(mask_pixel_pad.astype(int))

## Create Support mask

### Option 1: Execute if you want to create a new support mask with elliptical mask widget

If you really want to create a new support mask, execute next cell and then the "InteractiveCircleCoordinates"-widget

In [ ]:
# How many references do you have?
nr_ref = 10

# Setup coordinates (nr_ref + 1 coordinates, as there is always the object aperture)
support_coordinates = [
    [(((pos_pad.shape[-2])) // 2, (pos_pad.shape[-1]) // 2), 10, 10, 0] for k in range(nr_ref + 1)
]

# Widget to find the positions and sizes of the different apertures
print("Cover the object & reference apertures for each set of reconstructions that originates from the same reference with circles.")
print("Optimization: Change one circle parameter, calc phase retrieval image, compare with images reconstructed with old circle parameter. Repeat!")

# Create plot
holo = pos_pad.copy() * mask_bs_pad* (1 - mask_pixel_smooth_pad)

# Reconstruct
if heraldo is True:
    recon, _ = reconstruct_heraldo(holo,experimental_setup,prop_dist = prop_dist, phase = phase, angle = heraldo_rotation)
elif heraldo is False:
    recon = fth.reconstruct(
        fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
        * np.exp(1j * phase)
    )
recon = np.real(recon)
    
ds_ellipse = interactive.InteractiveEllipseCoordinates(recon,
    len(support_coordinates),
    coordinates=support_coordinates,
)

In [ ]:
# Take coordinates of circles from widget
support_coordinates = ds_ellipse.get_params()

# Create supportmask from coordinates
supportmask = create_ellipse_supportmask(support_coordinates, pos.shape)

# Add heraldo slit
#supportmask[1018:1030,880:1024] = 1

# Plot supportmask as overlay
fig, ax = plt.subplots(figsize=(6, 6))
mi, ma = np.percentile(recon, (1, 99))
ax.imshow(recon, vmin=mi, vmax=ma, cmap="gray")
ax.imshow(supportmask, alpha=0.4, cmap="binary")
ax.set_title("Image with overlayed mask")

### Option 2: Execute if you want to load an existing support mask created with elliptical mask widget

In [ ]:
reload(interactive)

In [ ]:
def get_supportmask_coordinates(sample):
    """
    Dictionary that stores coordinates of circular support mask apertures
    """

    # Setup dictonary
    support_coord = dict()

    # coordinates
    support_coord["s2304a_D5"] = [((409.0, 1696.0), 410.0, 410.0, 0.0), ((645.5, 1427.5), 24.0, 24.0, 0.0), ((475.5, 1394.5), 24.0, 24.0, 0.0), ((308.0, 1404.0), 24.0, 24.0, 0.0), ((674.3, 1934.5), 24.0, 24.0, 0.0), ((553.4, 2025.0), 24.0, 24.0, 0.0), ((424.0, 1990.0), 24.0, 24.0, 0.0), ((302.0, 2038.0), 24.0, 24.0, 0.0), ((173.0, 1962.0), 24.0, 24.0, 0.0), ((1273.0, 2068.0), 13.0, 13.0, 0.0), ((1224.5, 1224.0), 13.0, 13.0, 0.0)]
    support_coord["SWA-B31"] = [((1336.0, 935.0), 108.0, 108.0, 0.0), ((1445.6, 723.6), 25.0, 25.0, 0.0), ((1022.5, 601.0), 34.0, 34.0, 0.0), ((1024.0, 1024.0), 20.0, 20.0, 0.0)]
    support_coord["FBEb21_Co"] = [((873.5, 952.5), 84.5, 84.5, 0.0), ((828.1, 1022.0), 10.0, 10.0, 0.0), ((848.4, 1040.0), 13.0, 13.0, 0.0), ((886.0, 1034.5), 10.0, 10.0, 0.0), ((911.0, 1028.0), 12.0, 12.0, 0.0), ((1015.1, 864.8), 10.0, 10.0, 0.0), ((1024.0, 1024.0), 10.0, 10.0, 0.0)]
    support_coord["FBEb21_Co_2x2"] = [((361.7, 440.5), 86.0, 86.0, 0.0), ((316.2, 509.5), 12.0, 12.0, 0.0), ((337.1, 527.4), 14.0, 14.0, 0.0), ((373.9, 522.5), 11.0, 11.0, 0.0), ((398.4, 515.8), 14.0, 14.0, 0.0), ((503.2, 352.5), 11.0, 11.0, 0.0), ((512.0, 512.0), 12.0, 12.0, 0.0)]
    support_coord["FBEb21_Dy_2x2"] = [((269.0, 389.5), 139.5, 139.5, 0.0), ((185.9, 507.7), 20.0, 20.0, 0.0), ((221.9, 538.0), 15.0, 15.0, 0.0), ((286.5, 530.2), 15.0, 15.0, 0.0), ((325.0, 518.5), 16.0, 16.0, 0.0), ((497.5, 247.0), 15.0, 15.0, 0.0), ((512.0, 512.0), 15.0, 15.0, 0.0)]
    support_coord["SWA-B34"] = [((660.5, 717.5), 63.0, 63.0, 0.0), ((607.5, 630.5), 17.0, 17.0, 0.0), ((512.0, 512.0), 20.0, 20.0, 0.0)]
    support_coord["SWA-B31"] = [((667.0, 475.0), 97.5, 96.0, 0.0), ((720.5, 587.0), 23.5, 22.0, 0.0), ((731.5, 377.0), 23.0, 21.0, 0.0), ((512.0, 512.0), 11.5, 10.5, 0.0)]
    support_coord["SWA-B34-heraldo"] = [((488.5, 642.5), 53.0, 53.0, 0.0), ((435.0, 556.0), 9.0, 9.0, 0.0), ((340.5, 436.7), 15.0, 16.5, 36.0)]#,((340.5, 436.7), 15.0, 16.5, 36.0),((340.5, 436.7), 15.0, 16.5, 36.0)]
    return support_coord[sample]

In [ ]:
# Which supportmask to load? ("s2306a-C1", "s2308a-B1", ...)
sample = "s2304a_D5"

# Get coordinates and create supportmask
support_coordinates = get_supportmask_coordinates(sample)

In [ ]:
# Widget to find the positions and sizes of the different apertures
print("Cover the object & reference apertures for each set of reconstructions that originates from the same reference with circles.")
print("Optimization: Change one circle parameter, calc phase retrieval image, compare with images reconstructed with old circle parameter. Repeat!")

# Plotting image
holo = (pos_pad.copy()+neg_pad.copy()) * mask_bs_pad* (1 - mask_pixel_smooth_pad)

# Reconstruct
if heraldo is True:
    recon, _ = reconstruct_heraldo(holo,experimental_setup,prop_dist = prop_dist, phase = phase, angle = heraldo_rotation)
elif heraldo is False:
    recon = fth.reconstruct(
        fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
        * np.exp(1j * phase)
    )
recon = np.real(recon)

ds_ellipse = interactive.InteractiveEllipseCoordinates(recon,
    len(support_coordinates),
    coordinates=support_coordinates,
)

In [ ]:
# Take coordinates of circles from widget
support_coordinates = ds_ellipse.get_params()

# Create supportmask from coordinates
supportmask = create_ellipse_supportmask(support_coordinates, recon.shape)

# Plotting image
holo = pos_pad.copy() * mask_bs_pad* (1 - mask_pixel_smooth_pad)
recon = np.real(recon)


# Plot supportmask as overlay
fig, ax = plt.subplots(figsize=(6, 6))
mi, ma = np.percentile(recon, (1, 99))
ax.imshow(recon, vmin=mi, vmax=ma, cmap="gray")
ax.imshow(supportmask, alpha=0.4, cmap="binary")
ax.set_title("Image with overlayed mask")

### Take Roi
Choose the reconstructions as the ROI.

1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
fig, ax = cimshow(supportmask.astype(int))

## Do Phase Retrieval

In [ ]:
# Executes the algorithm
offset_vmin = .5
recon = fth.reconstruct(
    fth.propagate((pos_pad+neg_pad), prop_dist * 1e-6, experimental_setup=experimental_setup)
    * np.exp(1j * phase)
)
Startimage = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(supportmask*recon)))
Startimage = None
Startgamma = None

(
    retrieved_p,
    retrieved_n,
    retrieved_p_pc,
    retrieved_n_pc,
    bsmask_p,
    bsmask_n,
    gamma_p,
    gamma_n,
) = phase_retrieval(pos_pad, neg_pad, mask_pixel_pad.astype(int), supportmask, vmin = offset_vmin, Startimage=Startimage, Startgamma=Startgamma)

## Reconstruct images from phase retrieval

In [ ]:
# New beamstop for CDI recos as phase retrieval of low-q might be insufficient. If phase retrieval worked well
# Try without beamstop: `use_bs = False`
use_bs = True
bs_diam_cdi = 65  # diameter of beamstop

# Create beamstop
if use_bs is True:
    mask_bs_cdi = 1 - cci.circle_mask(
        recon.shape, np.array(recon.shape) / 2, bs_diam_cdi, sigma=4
    )
    mask_bs_cdi = 1 - mask_pixel_smooth_pad.copy()
elif use_bs is False:
    mask_bs_cdi = np.ones(recon.shape)  # if you don't want a beamstop

# Get Recos
# Positive helicity
p = cci.reconstruct(
    fth.propagate(
        retrieved_p * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)

# Negative helicity
n = cci.reconstruct(
    fth.propagate(
        retrieved_n * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)

tshift = [-2,1.75]
tretrieved_p_pc = cci.shift_image(retrieved_p_pc,tshift)
tretrieved_n_pc = cci.shift_image(retrieved_n_pc,tshift)

# Get Recos partial coherence
# Positiv partial coherence
p_pc = cci.reconstruct(
    fth.propagate(
        tretrieved_p_pc * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)

# Negative partial coherence
n_pc = cci.reconstruct(
    fth.propagate(
        tretrieved_n_pc * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)

# optimize phase
recon = p_pc - n_pc
#_, phase_cdi = optimize_phase_contrast(recon, supportmask, method="contrast",prefered_color="white")

# Plotting
mode = "+"
print("Fine-tuning of reconstruction parameter:")
slider_prop, slider_phase, slider_dx, slider_dy = rec.focusCDI(
    tretrieved_p_pc * mask_bs_cdi,
    tretrieved_n_pc * mask_bs_cdi,
    #retrieved_p * mask_bs_cdi,
    #retrieved_n * mask_bs_cdi,
    roi_cdi,
    mask=supportmask,
    phase=phase_cdi,
    dx=dx,
    dy=dy,
    prop_dist=prop_dist_cdi,
    experimental_setup=experimental_setup,
    operation=mode,
    max_prop_dist=20,
    scale=(1, 99),
)

In [ ]:
# Get phase from slider
phase_cdi = slider_phase.value
prop_dist_cdi = slider_prop.value

# Reconstruct images with new parameter
p_pc = fth.reconstructCDI(
    fth.propagate(
        tretrieved_p_pc * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
) * np.exp(1j * phase_cdi)

n_pc = fth.reconstructCDI(
    fth.propagate(
        tretrieved_n_pc * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
) * np.exp(1j * phase_cdi)


print("Phase CDI: %s" % phase_cdi)
print("Prop_dist: %s" % prop_dist_cdi)

In [ ]:
cimshow(np.real(p_pc+n_pc)*supportmask,cmap="gray")

In [ ]:
# Confirm that offset subtraction in cdi function works, i.e., only small fraction of hologram is actually masked
fig, ax = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)
tmp = np.abs(retrieved_p_pc*mask_bs_cdi)
mi, ma = np.percentile(tmp, [0.1, 99.9])
ax[0, 0].imshow(tmp, vmin=mi, vmax=ma)
ax[0, 0].set_title("Pos holo")

tmp = np.abs(retrieved_n_pc)
mi, ma = np.percentile(tmp, [0.1, 99.9])
ax[0, 1].imshow(tmp, vmin=mi, vmax=ma)
ax[0, 1].set_title("Neg holo")
ax[1, 0].imshow(bsmask_p)
ax[1, 0].set_title("Pos holo mask")
ax[1, 1].imshow(bsmask_n)
ax[1, 1].set_title("Neg holo mask")

In [ ]:
# Ta-ke initial values
# Open figure
fig, [ax, ax2,ax3] = plt.subplots(1, 3, figsize=(12, 4))

# Widget
layout = ipywidgets.Layout(width="100%")
opts = dict(layout=layout)


@ipywidgets.interact(
    centery=(center[0] - 10, center[0] + 10, .25),
    centerx=(center[1] - 10, center[1] + 10, .25),
    tphase=(-np.pi,np.pi,.2),
    options=opts,
)
def update(
    centery=center[0],
    centerx=center[1],
    tphase=phase,
):  # initial values
    global temp_center, temp_phase  # ugly writing as global variable

    # Couple widget only and global variables
    temp_center = [centery, centerx]
    temp_phase = tphase

    temp_diff = cci.shift_image(
        retrieved_p_pc-retrieved_n_pc, -np.array([centery - center[0], centerx - center[1]])
    )
    temp_sum = cci.shift_image(
        retrieved_p_pc+retrieved_n_pc, -np.array([centery - center[0], centerx - center[1]])
    )

    temp_diff_b = temp_diff #* mask_bs * (1 - mask_pixel_smooth)
    temp_sum_b = temp_sum * (1 - mask_pixel_smooth_pad)
    
    # Reconstruction
    recon = fth.reconstructCDI(
        fth.propagate(
            temp_sum_b,
            prop_dist * 1e-6,
            experimental_setup=experimental_setup,
        )
        * np.exp(1j * temp_phase)
    )[np.s_[1450:1950,150:650]]

    # Plots
    vmin, vmax = np.percentile(np.real(recon), (1, 99))
    ax.imshow(np.real(recon), vmin=vmin, vmax=vmax, cmap="gray")
    vmin, vmax = np.percentile(np.imag(recon), (1, 99))
    ax2.imshow(np.imag(recon), vmin=vmin, vmax=vmax, cmap="gray")

    
    # Add circles with different radi r
    vmin, vmax = np.percentile(temp_sum, (0.1, 99.999))
    ax3.imshow(np.abs(temp_sum), vmin=vmin, vmax=vmax)
    ax3.set_xlim([1000,1400])
    ax3.set_ylim([1000,1400])
    for r in np.arange(0, 150, 25):
        ax3.add_artist(plt.Circle(np.array(mask_bs_pad.shape)/2, r, fill=None, ec="red"))

## Save reconstructions

In [ ]:
def plot_recon(recon, title):
    # Plot
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    fig.suptitle(title)

    vmin, vmax = np.percentile(np.real(recon), (2, 98))
    t_im1 = ax[0].imshow(np.real(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[0].set_title("Real")
    plt.colorbar(t_im1, ax=ax[0], aspect=50)

    vmin, vmax = np.percentile(np.imag(recon), (2, 98))
    t_im2 = ax[1].imshow(np.imag(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[1].set_title("Imag")
    plt.colorbar(t_im2, ax=ax[1], aspect=50)

In [ ]:
# Saves only real and imaginary part
#recon = p - n
recon = (p_pc + n_pc)#   / (p_pc + n_pc)

# Plot
title = get_title("srotz", im_id,topo_id, CDI=True)
#plot_recon(np.rot90(recon,-2)[roi_cdi], title)
plot_recon(recon[roi_cdi], title)

# Save images
fname = join(
    folder_general,
    "Recon_ImId_%04d_RefId_%s_cdi_sum_%s.png" % (im_id, topo_id, USER),
)
print("Saving: %s" % fname)
plt.savefig(fname, bbox_inches="tight", transparent=False)

# Save h5
save_cdi_h5()

In [ ]:
def plot_recon(recon, title):
    # Plot
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    fig.suptitle(title)

    vmin, vmax = np.percentile(np.real(recon), (.1, 99.9))
    t_im1 = ax[0].imshow(np.real(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[0].set_title("Real")
    plt.colorbar(t_im1, ax=ax[0], aspect=50)

    vmin, vmax = np.percentile(np.imag(recon), (.1, 99.9))
    t_im2 = ax[1].imshow(np.imag(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[1].set_title("Imag")
    plt.colorbar(t_im2, ax=ax[1], aspect=50)

In [ ]:
# Saves only real and imaginary part
#recon = p - n
recon = (p_pc - n_pc)#   / (p_pc + n_pc)

# Plot
title = get_title("srotz", im_id,topo_id, CDI=True)
#plot_recon(np.rot90(recon,-2)[roi_cdi], title)
plot_recon(recon[roi_cdi], title)

# Save images
fname = join(
    folder_general,
    "Recon_ImId_%04d_RefId_%s_cdi_diff_%s.png" % (im_id, topo_id, USER),
)
print("Saving: %s" % fname)
plt.savefig(fname, bbox_inches="tight", transparent=False)

# Save h5
save_cdi_h5()

In [ ]:
cimshow(neg)